# Challenge TelecomX - Parte 2: Modelos Preditivos

## Objetivo
Criar modelos de Machine Learning para prever a evasão de clientes (churn) da TelecomX, utilizando os dados já tratados na Parte 1 do desafio.

## Etapas do Projeto
1. **Carregamento dos dados tratados**
2. **Análise exploratória para modelagem**
3. **Preparação dos dados para ML**
4. **Construção de modelos preditivos**
5. **Avaliação e comparação dos modelos**
6. **Interpretação dos resultados**

In [ ]:
%pip install scikit-learn pandas numpy matplotlib seaborn imbalanced-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek

print("Bibliotecas importadas com sucesso!")

: 

## 1. Carregamento dos Dados Tratados

Carregando o conjunto de dados que foi limpo e tratado na Parte 1 do Challenge TelecomX.

In [ ]:
try:
    dados = pd.read_csv("dados_tratados.csv")
    print("Dados tratados carregados do arquivo CSV")
except FileNotFoundError:
    print("Arquivo dados_tratados.csv não encontrado")
    print("Carregando e tratando dados da API...")
    
    import requests
    
    url = 'https://github.com/alura-cursos/challenge2-data-science/raw/refs/heads/main/TelecomX_Data.json'
    response = requests.get(url)
    data = response.json()
    
    dados = pd.json_normalize(data)
    dados.replace({'Yes': 1, 'No': 0}, inplace=True)
    
    colunas_numericas = ['account.Charges.Monthly', 'account.Charges.Total', 'customer.tenure']
    for col in colunas_numericas:
        if col in dados.columns:
            dados[col] = pd.to_numeric(dados[col], errors='coerce')
    
    if 'account.Charges.Monthly' in dados.columns:
        dados['Contas_Diarias'] = dados['account.Charges.Monthly'] / 30
    
    dados = dados[dados['Churn'].isin([0, 1])]
    
    print("Dados carregados e tratados da API")

print(f"Dimensões do dataset: {dados.shape}")
print(f"Colunas disponíveis: {len(dados.columns)}")
dados.head()

In [ ]:
print("Informações do dataset:")
print(f"Registros: {len(dados):,}")
print(f"Colunas: {dados.columns.tolist()}")
print(f"\nTipos de dados:")
print(dados.dtypes)
print(f"\nValores ausentes:")
print(dados.isnull().sum().sum())

if 'Churn' in dados.columns:
    print(f"\nDistribuição do churn:")
    churn_counts = dados['Churn'].value_counts()
    print(f"Não cancelaram (0): {churn_counts[0]:,} ({churn_counts[0]/len(dados)*100:.1f}%)")
    print(f"Cancelaram (1): {churn_counts[1]:,} ({churn_counts[1]/len(dados)*100:.1f}%)")
    
    plt.figure(figsize=(6,4))
    dados['Churn'].value_counts().plot(kind='bar', color=['lightblue', 'coral'])
    plt.title('Distribuição da Variável Target (Churn)')
    plt.xlabel('Churn')
    plt.ylabel('Quantidade')
    plt.xticks([0,1], ['Não Cancelou', 'Cancelou'], rotation=0)
    plt.tight_layout()
    plt.show()

## 2. Pré-processamento Avançado dos Dados

### Etapas do Pré-processamento:
1. **Eliminação de colunas irrelevantes** (IDs, identificadores únicos)
2. **Codificação de variáveis categóricas** (One-Hot Encoding)
3. **Análise do balanceamento das classes**
4. **Aplicação de técnicas de balanceamento** (SMOTE, undersampling)
5. **Normalização/Padronização** conforme necessário para cada modelo

In [ ]:
# Etapa 1: Eliminação de Colunas Irrelevantes

print("Análise das colunas do dataset:")

print("Colunas disponíveis:")
for i, col in enumerate(dados.columns):
    print(f"{i+1:2d}. {col}")
    
print(f"\nShape original dos dados: {dados.shape}")

colunas_para_remover = []

for col in dados.columns:
    if 'id' in col.lower() or 'customerid' in col.lower():
        colunas_para_remover.append(col)
        print(f"Coluna '{col}' identificada como ID")
    elif dados[col].nunique() == len(dados):
        colunas_para_remover.append(col)
        print(f"Coluna '{col}' tem valores únicos (possível ID)")

if colunas_para_remover:
    dados_limpos = dados.drop(columns=colunas_para_remover)
    print(f"Colunas removidas: {colunas_para_remover}")
    print(f"Shape após remoção: {dados_limpos.shape}")
else:
    dados_limpos = dados.copy()
    print("Nenhuma coluna irrelevante identificada automaticamente")

print("\nColunas mantidas:")
for col in dados_limpos.columns:
    tipo_dado = dados_limpos[col].dtype
    valores_unicos = dados_limpos[col].nunique()
    print(f"• {col:20s} | Tipo: {str(tipo_dado):10s} | Únicos: {valores_unicos:4d}")

print(f"\nDados prontos para próxima etapa: {dados_limpos.shape}")

In [ ]:
# Etapa 2: Codificação de Variáveis Categóricas

print("Transformando variáveis categóricas")

colunas_categoricas = []
colunas_numericas = []
coluna_target = None

for col in dados_limpos.columns:
    if 'churn' in col.lower() or 'evasao' in col.lower() or 'cancelou' in col.lower():
        coluna_target = col
        print(f"Variável target identificada: '{col}'")
    elif dados_limpos[col].dtype == 'object' or dados_limpos[col].dtype.name == 'category':
        colunas_categoricas.append(col)
    elif pd.api.types.is_numeric_dtype(dados_limpos[col]):
        colunas_numericas.append(col)

print(f"\nResumo das variáveis:")
print(f"• Categóricas: {len(colunas_categoricas)} - {colunas_categoricas}")
print(f"• Numéricas: {len(colunas_numericas)} - {colunas_numericas}")
print(f"• Target: {coluna_target}")

if coluna_target:
    X = dados_limpos.drop(coluna_target, axis=1)
    y = dados_limpos[coluna_target]
else:
    X = dados_limpos.iloc[:, :-1]
    y = dados_limpos.iloc[:, -1]
    coluna_target = dados_limpos.columns[-1]

X_numericas = X.select_dtypes(include=[np.number])
X_categoricas = X.select_dtypes(include=['object', 'category'])

print(f"\nSeparação concluída:")
print(f"X (features): {X.shape}")
print(f"y (target): {y.shape}")

if not X_categoricas.empty:
    print(f"\nAplicando One-Hot Encoding em {len(X_categoricas.columns)} variáveis categóricas...")
    
    # Converter todas as colunas categóricas para string para evitar erro de tipos mistos
    X_categoricas_str = X_categoricas.astype(str)
    
    encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    X_categoricas_encoded = encoder.fit_transform(X_categoricas_str)
    
    nomes_encoded = encoder.get_feature_names_out(X_categoricas.columns)
    X_categoricas_encoded = pd.DataFrame(X_categoricas_encoded, 
                                       columns=nomes_encoded, 
                                       index=X_categoricas.index)
    
    X_final = pd.concat([X_numericas, X_categoricas_encoded], axis=1)
    
    print(f"One-Hot Encoding aplicado")
    print(f"Shape antes: {X.shape} | Shape depois: {X_final.shape}")
    print(f"Novas colunas criadas: {len(nomes_encoded)}")
    
else:
    X_final = X.copy()
    print("Nenhuma variável categórica encontrada - dados já numéricos")

if y.dtype == 'object':
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    print(f"\nTarget '{coluna_target}' codificado:")
    for i, classe in enumerate(label_encoder.classes_):
        print(f"   • {classe} → {i}")
else:
    y_encoded = y.copy()
    print(f"\nTarget '{coluna_target}' já está em formato numérico")

print(f"\nDados finais para modelagem:")
print(f"• Features (X): {X_final.shape}")
print(f"• Target (y): {y_encoded.shape}")
print(f"• Colunas finais: {list(X_final.columns)}")

In [ ]:
# Etapa 3: Análise do Balanceamento das Classes

print("Análise de balanceamento das classes")

distribuicao_classes = pd.Series(y_encoded).value_counts().sort_index()
total_amostras = len(y_encoded)

print("Distribuição absoluta:")
for classe, quantidade in distribuicao_classes.items():
    print(f"   Classe {classe}: {quantidade:,} amostras")

print(f"\nDistribuição percentual:")
for classe, quantidade in distribuicao_classes.items():
    percentual = (quantidade / total_amostras) * 100
    print(f"   Classe {classe}: {percentual:.2f}%")

classe_majoritaria = distribuicao_classes.max()
classe_minoritaria = distribuicao_classes.min()
razao_desbalanceamento = classe_majoritaria / classe_minoritaria

print(f"\nMétricas de balanceamento:")
print(f"   • Classe majoritária: {classe_majoritaria:,} amostras")
print(f"   • Classe minoritária: {classe_minoritaria:,} amostras")
print(f"   • Razão de desbalanceamento: {razao_desbalanceamento:.2f}:1")

if razao_desbalanceamento <= 1.5:
    nivel_desbalanceamento = "BAIXO"
elif razao_desbalanceamento <= 3.0:
    nivel_desbalanceamento = "MODERADO"
elif razao_desbalanceamento <= 10.0:
    nivel_desbalanceamento = "ALTO"
else:
    nivel_desbalanceamento = "MUITO ALTO"

print(f"   • Nível de desbalanceamento: {nivel_desbalanceamento}")

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
distribuicao_classes.plot(kind='bar', color=['skyblue', 'lightcoral'])
plt.title('Distribuição das Classes', fontsize=14, fontweight='bold')
plt.xlabel('Classe', fontsize=12)
plt.ylabel('Quantidade de Amostras', fontsize=12)
plt.xticks(rotation=0)
for i, v in enumerate(distribuicao_classes.values):
    plt.text(i, v + total_amostras*0.01, f'{v:,}', ha='center', fontweight='bold')

plt.subplot(1, 2, 2)
labels = [f'Classe {i}\n({v:,} amostras)' for i, v in enumerate(distribuicao_classes.values)]
colors = ['lightblue', 'lightcoral']
plt.pie(distribuicao_classes.values, labels=labels, autopct='%1.1f%%', 
        colors=colors, startangle=90, textprops={'fontsize': 10})
plt.title('Proporção das Classes', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nRecomendações para {nivel_desbalanceamento} desbalanceamento:")
if razao_desbalanceamento <= 1.5:
    print("   Dataset bem balanceado - modelos podem ser treinados sem ajustes especiais")
elif razao_desbalanceamento <= 3.0:
    print("   Considerar ajuste de class_weight nos modelos")
    print("   Monitorar métricas de precision e recall por classe")
elif razao_desbalanceamento <= 10.0:
    print("   Recomendado aplicar técnicas de balanceamento (SMOTE, undersampling)")
    print("   Usar métricas balanceadas (F1-score, ROC-AUC) para avaliação")
    print("   Ajustar threshold de classificação se necessário")
else:
    print("   OBRIGATÓRIO aplicar técnicas de balanceamento")
    print("   Considerar estratégias ensemble especializadas")
    print("   Foco em recall da classe minoritária")

print(f"\nAnálise de balanceamento concluída")

In [ ]:
# Etapa 4: Aplicação de Técnicas de Balanceamento

print("Aplicando técnicas de balanceamento")

datasets_balanceados = {}
datasets_balanceados['Original'] = (X_final, y_encoded)

if razao_desbalanceamento > 1.5:
    print(f"Aplicando técnicas de balanceamento (razão: {razao_desbalanceamento:.2f}:1)")
    
    try:
        print("\nAplicando SMOTE...")
        smote = SMOTE(random_state=42, k_neighbors=min(5, classe_minoritaria-1))
        X_smote, y_smote = smote.fit_resample(X_final, y_encoded)
        datasets_balanceados['SMOTE'] = (X_smote, y_smote)
        print(f"   SMOTE aplicado - Shape: {X_smote.shape}")
        
    except Exception as e:
        print(f"   Erro no SMOTE: {e}")
    
    try:
        print("\nAplicando Random Under Sampling...")
        undersampler = RandomUnderSampler(random_state=42)
        X_under, y_under = undersampler.fit_resample(X_final, y_encoded)
        datasets_balanceados['UnderSampling'] = (X_under, y_under)
        print(f"   Under Sampling aplicado - Shape: {X_under.shape}")
        
    except Exception as e:
        print(f"   Erro no Under Sampling: {e}")
    
    try:
        print("\nAplicando SMOTE + Tomek...")
        smote_tomek = SMOTETomek(random_state=42)
        X_smote_tomek, y_smote_tomek = smote_tomek.fit_resample(X_final, y_encoded)
        datasets_balanceados['SMOTE_Tomek'] = (X_smote_tomek, y_smote_tomek)
        print(f"   SMOTE+Tomek aplicado - Shape: {X_smote_tomek.shape}")
        
    except Exception as e:
        print(f"   Erro no SMOTE+Tomek: {e}")

else:
    print(f"Dataset já bem balanceado (razão: {razao_desbalanceamento:.2f}:1) - balanceamento não necessário")

print(f"\nComparação das distribuições:")

for nome, (X_data, y_data) in datasets_balanceados.items():
    distribuicao = pd.Series(y_data).value_counts().sort_index()
    total = len(y_data)
    razao = distribuicao.max() / distribuicao.min() if len(distribuicao) > 1 else 1.0
    
    print(f"\n{nome.upper()}:")
    print(f"   Total de amostras: {total:,}")
    for classe, qtd in distribuicao.items():
        perc = (qtd/total)*100
        print(f"   Classe {classe}: {qtd:,} ({perc:.1f}%)")
    print(f"   Razão de balanceamento: {razao:.2f}:1")

if len(datasets_balanceados) > 1:
    fig, axes = plt.subplots(2, len(datasets_balanceados), figsize=(4*len(datasets_balanceados), 8))
    if len(datasets_balanceados) == 1:
        axes = axes.reshape(-1, 1)
    
    for i, (nome, (X_data, y_data)) in enumerate(datasets_balanceados.items()):
        dist = pd.Series(y_data).value_counts().sort_index()
        axes[0, i].bar(range(len(dist)), dist.values, color=['skyblue', 'lightcoral'])
        axes[0, i].set_title(f'{nome}\n({len(y_data):,} amostras)', fontsize=12, fontweight='bold')
        axes[0, i].set_xlabel('Classe')
        axes[0, i].set_ylabel('Quantidade')
        
        for j, v in enumerate(dist.values):
            axes[0, i].text(j, v + max(dist.values)*0.01, f'{v:,}', ha='center', fontweight='bold')
        
        axes[1, i].pie(dist.values, labels=[f'Classe {j}' for j in range(len(dist))], 
                      autopct='%1.1f%%', startangle=90)
        axes[1, i].set_title(f'Proporção - {nome}')
    
    plt.tight_layout()
    plt.show()

dataset_escolhido = 'SMOTE' if 'SMOTE' in datasets_balanceados else 'Original'
X_balanceado, y_balanceado = datasets_balanceados[dataset_escolhido]

print(f"\nDataset escolhido para modelagem: {dataset_escolhido}")
print(f"Shape final: X={X_balanceado.shape}, y={y_balanceado.shape}")
print("Técnicas de balanceamento aplicadas com sucesso")

## 3. Análise de Correlação e Seleção de Variáveis

### Objetivos:
1. **Visualizar matriz de correlação** entre todas as variáveis
2. **Identificar correlações com o target** (variáveis mais relevantes)
3. **Investigar relações específicas** entre tempo de contrato e evasão
4. **Analisar impacto dos gastos** na decisão de cancelamento
5. **Selecionar features mais relevantes** para modelagem

In [ ]:
# Etapa 1: Divisão Estratificada dos Dados

print("Divisão dos dados em treino e teste")

try:
    if 'features_recomendadas' in locals() and 'X_final' in locals() and 'y_encoded' in locals():
        X_modelagem = X_final[features_recomendadas[:15]]
        y_modelagem = y_encoded
        print("Usando features selecionadas da análise de correlação")
        print(f"Features utilizadas ({len(X_modelagem.columns)}): {list(X_modelagem.columns)}")
    else:
        X_modelagem = X_final
        y_modelagem = y_encoded
        print("Usando todas as features processadas disponíveis")
        
except NameError:
    print("Executando pré-processamento básico...")
    
    if 'Churn' in dados.columns:
        X_modelagem = dados.drop('Churn', axis=1)
        y_modelagem = dados['Churn']
        coluna_target = 'Churn'
    else:
        X_modelagem = dados.iloc[:, :-1]
        y_modelagem = dados.iloc[:, -1]
        coluna_target = dados.columns[-1]
    
    for col in X_modelagem.select_dtypes(include=['object']).columns:
        if col != 'customerID':
            le = LabelEncoder()
            X_modelagem[col] = le.fit_transform(X_modelagem[col].astype(str))
    
    id_columns = [col for col in X_modelagem.columns if 'id' in col.lower()]
    if id_columns:
        X_modelagem = X_modelagem.drop(columns=id_columns)
        print(f"Removidas colunas ID: {id_columns}")
    
    print("Pré-processamento básico aplicado")

print(f"\nDimensões dos dados:")
print(f"   • Features (X): {X_modelagem.shape}")
print(f"   • Target (y): {y_modelagem.shape}")
print(f"   • Total de amostras: {len(y_modelagem):,}")

distribuicao_original = pd.Series(y_modelagem).value_counts()
print(f"\nDistribuição original das classes:")
for classe, qtd in distribuicao_original.items():
    percentual = (qtd / len(y_modelagem)) * 100
    print(f"   • Classe {classe}: {qtd:,} amostras ({percentual:.1f}%)")

divisoes = [
    {'test_size': 0.2, 'nome': '80/20', 'recomendacao': 'PADRÃO'},
    {'test_size': 0.3, 'nome': '70/30', 'recomendacao': 'ALTERNATIVA'}
]

print(f"\nTestando diferentes divisões:")

datasets_divididos = {}

for config in divisoes:
    X_train, X_test, y_train, y_test = train_test_split(
        X_modelagem, y_modelagem,
        test_size=config['test_size'],
        random_state=42,
        stratify=y_modelagem
    )
    
    datasets_divididos[config['nome']] = {
        'X_train': X_train, 'X_test': X_test,
        'y_train': y_train, 'y_test': y_test,
        'config': config
    }
    
    print(f"\nDivisão {config['nome']} ({config['recomendacao']}):")
    print(f"   • Treino: {X_train.shape[0]:,} amostras ({(1-config['test_size'])*100:.0f}%)")
    print(f"   • Teste:  {X_test.shape[0]:,} amostras ({config['test_size']*100:.0f}%)")
    
    dist_treino = pd.Series(y_train).value_counts(normalize=True)
    dist_teste = pd.Series(y_test).value_counts(normalize=True)
    
    print("   Distribuição Treino vs Teste:")
    for classe in dist_treino.index:
        diff = abs(dist_treino[classe] - dist_teste[classe]) * 100
        status = "OK" if diff < 2 else "Alerta"
        print(f"      {status} Classe {classe}: Treino {dist_treino[classe]:.1%} | Teste {dist_teste[classe]:.1%} | Diff: {diff:.1f}pp")

divisao_escolhida = '80/20'
dados_finais = datasets_divididos[divisao_escolhida]

X_train = dados_finais['X_train']
X_test = dados_finais['X_test']
y_train = dados_finais['y_train'] 
y_test = dados_finais['y_test']

print(f"\nDivisão escolhida: {divisao_escolhida}")
print(f"Dados preparados para modelagem:")
print(f"   • X_train: {X_train.shape}")
print(f"   • X_test: {X_test.shape}")
print(f"   • y_train: {y_train.shape}")
print(f"   • y_test: {y_test.shape}")

print(f"\nDivisão estratificada dos dados concluída")
print("Dados prontos para treinamento dos modelos")

In [ ]:
# Etapa 2: Preparação Específica por Tipo de Modelo

print("Preparação dos dados por categoria de modelo")

categorias_modelos = {
    'Sensíveis à Escala': {
        'modelos': ['Logistic Regression', 'KNN'],
        'precisa_normalizacao': True,
        'justificativa': [
            'Regressão Logística usa gradiente descendente (sensível a diferentes escalas)',
            'KNN calcula distâncias euclidianas (features com escalas maiores dominam)',
            'Convergência mais rápida e estável com dados normalizados',
            'Coeficientes mais interpretáveis na Regressão Logística'
        ]
    },
    'Baseados em Árvore': {
        'modelos': ['Decision Tree', 'Random Forest'],
        'precisa_normalizacao': False,
        'justificativa': [
            'Árvores fazem divisões baseadas em valores absolutos (thresholds)',
            'Algoritmo é invariante a transformações monótonas',
            'Cada feature é avaliada independentemente',
            'Normalização não afeta a estrutura das decisões'
        ]
    }
}

print("Justificativas técnicas por categoria:")

for categoria, info in categorias_modelos.items():
    necessidade = "REQUER" if info['precisa_normalizacao'] else "NÃO REQUER"
    
    print(f"\n{categoria.upper()} - {necessidade} NORMALIZAÇÃO:")
    print(f"   • Modelos: {', '.join(info['modelos'])}")
    print("   • Justificativas técnicas:")
    for just in info['justificativa']:
        print(f"     ○ {just}")

datasets_por_categoria = {}

print(f"\nPreparando datasets específicos:")

print("\nCATEGORIA: Modelos Baseados em Árvore")
print("   Estratégia: Dados originais (sem normalização)")

datasets_por_categoria['Baseados em Árvore'] = {
    'X_train': X_train.copy(),
    'X_test': X_test.copy(),
    'y_train': y_train.copy(),
    'y_test': y_test.copy(),
    'scaler': None,
    'normalizacao_aplicada': False
}

print("   Dataset preparado (dados mantidos na escala original)")

print("\nCATEGORIA: Modelos Sensíveis à Escala")
print("   Estratégia: StandardScaler (média=0, std=1)")

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

datasets_por_categoria['Sensíveis à Escala'] = {
    'X_train': X_train_scaled,
    'X_test': X_test_scaled,
    'y_train': y_train.copy(),
    'y_test': y_test.copy(),
    'scaler': scaler,
    'normalizacao_aplicada': True
}

print("   StandardScaler aplicado")

print(f"\nComparando escalas antes e depois:")

# Amostra de features para comparação
features_amostra = X_train.columns[:5]

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Antes da normalização
axes[0,0].boxplot([X_train[col] for col in features_amostra], labels=features_amostra)
axes[0,0].set_title('ANTES - Escalas Originais')
axes[0,0].set_ylabel('Valor')
axes[0,0].tick_params(axis='x', rotation=45)
axes[0,0].grid(alpha=0.3)

# Depois da normalização
axes[0,1].boxplot([X_train_scaled[col] for col in features_amostra], labels=features_amostra)
axes[0,1].set_title('DEPOIS - Escalas Padronizadas')
axes[0,1].set_ylabel('Valor (Z-score)')
axes[0,1].tick_params(axis='x', rotation=45)
axes[0,1].grid(alpha=0.3)

# Histograma antes
for i, col in enumerate(features_amostra[:2]):
    axes[1,0].hist(X_train[col], alpha=0.7, bins=20, label=col)
axes[1,0].set_title('ANTES - Distribuições Originais')
axes[1,0].set_xlabel('Valor')
axes[1,0].set_ylabel('Frequência')
axes[1,0].legend()
axes[1,0].grid(alpha=0.3)

# Histograma depois
for i, col in enumerate(features_amostra[:2]):
    axes[1,1].hist(X_train_scaled[col], alpha=0.7, bins=20, label=col)
axes[1,1].set_title('DEPOIS - Escalas Padronizadas')
axes[1,1].set_ylabel('Valor (Z-score)')
axes[1,1].tick_params(axis='x', rotation=45)
axes[1,1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nResumo dos datasets preparados:")

for categoria, dados in datasets_por_categoria.items():
    print(f"\n{categoria.upper()}:")
    print(f"   • Normalização: {'SIM' if dados['normalizacao_aplicada'] else 'NÃO'}")
    print(f"   • X_train shape: {dados['X_train'].shape}")
    print(f"   • X_test shape: {dados['X_test'].shape}")
    print(f"   • Modelos compatíveis: {', '.join(categorias_modelos[categoria]['modelos'])}")
    if dados['scaler']:
        print(f"   • Scaler: {type(dados['scaler']).__name__}")

print(f"\nPreparação específica por tipo de modelo concluída")
print("Datasets otimizados prontos para treinamento")

In [ ]:
# Etapa 3: Treinamento dos Modelos Selecionados

print("Treinamento dos modelos preditivos")

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import time

modelos_config = {
    'Regressão Logística': {
        'modelo': LogisticRegression(random_state=42, max_iter=1000),
        'categoria': 'Sensíveis à Escala',
        'dataset_key': 'Sensíveis à Escala',
        'justificativa': [
            'Modelo linear baseado em probabilidades (função sigmoide)',
            'Coeficientes interpretáveis para análise de impacto',
            'Computacionalmente eficiente para datasets grandes',
            'Baseline robusto para problemas de classificação binária'
        ],
        'vantagens': ['Interpretabilidade alta', 'Rápido treinamento', 'Probabilidades calibradas'],
        'quando_usar': 'Relações lineares entre features e target'
    },
    
    'K-Nearest Neighbors': {
        'modelo': KNeighborsClassifier(n_neighbors=5, metric='euclidean'),
        'categoria': 'Sensíveis à Escala',
        'dataset_key': 'Sensíveis à Escala',
        'justificativa': [
            'Algoritmo não-paramétrico baseado em vizinhança',
            'Captura padrões locais complexos nos dados',
            'Flexível para fronteiras de decisão não-lineares',
            'Não assume distribuição específica dos dados'
        ],
        'vantagens': ['Sem suposições sobre dados', 'Fronteiras complexas', 'Simples conceitualmente'],
        'quando_usar': 'Padrões locais complexos, dados com clusters naturais'
    },
    
    'Árvore de Decisão': {
        'modelo': DecisionTreeClassifier(random_state=42, max_depth=10, min_samples_split=20),
        'categoria': 'Baseados em Árvore',
        'dataset_key': 'Baseados em Árvore',
        'justificativa': [
            'Modelo de regras interpretáveis (if-then-else)',
            'Captura interações não-lineares entre features',
            'Seleção automática de features mais importantes',
            'Não afetado por outliers e diferentes escalas'
        ],
        'vantagens': ['Máxima interpretabilidade', 'Sem pré-processamento', 'Captura interações'],
        'quando_usar': 'Necessidade de explicabilidade total das decisões'
    },
    
    'Random Forest': {
        'modelo': RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10),
        'categoria': 'Baseados em Árvore',
        'dataset_key': 'Baseados em Árvore',
        'justificativa': [
            'Ensemble de árvores reduz overfitting individual',
            'Robustez através de bootstrap aggregating (bagging)',
            'Feature importance automática e confiável',
            'Excelente performance geral na maioria dos problemas'
        ],
        'vantagens': ['Alta precisão', 'Robusto a overfitting', 'Feature importance'],
        'quando_usar': 'Quando performance é prioridade sobre interpretabilidade'
    }
}

print("Justificativas detalhadas para cada modelo:")

for nome_modelo, config in modelos_config.items():
    necessidade = "REQUER" if config['categoria'] == 'Sensíveis à Escala' else "NÃO REQUER"
    
    print(f"\n{nome_modelo.upper()} ({config['categoria']}) - {necessidade} NORMALIZAÇÃO:")
    print("   Justificativas técnicas:")
    for just in config['justificativa']:
        print(f"      • {just}")
    print(f"   Vantagens: {', '.join(config['vantagens'])}")
    print(f"   Quando usar: {config['quando_usar']}")

resultados_modelos = {}

print(f"\nIniciando treinamento dos modelos...")

for nome_modelo, config in modelos_config.items():
    print(f"\nTreinando: {nome_modelo}")
    
    dataset = datasets_por_categoria[config['dataset_key']]
    X_train_modelo = dataset['X_train'].copy()
    X_test_modelo = dataset['X_test'].copy()
    y_train_modelo = dataset['y_train']
    y_test_modelo = dataset['y_test']
    
    # Verificar e tratar valores NaN
    nan_count_train = X_train_modelo.isnull().sum().sum()
    nan_count_test = X_test_modelo.isnull().sum().sum()
    
    if nan_count_train > 0 or nan_count_test > 0:
        print(f"   Valores NaN encontrados: Train={nan_count_train}, Test={nan_count_test}")
        print("   Aplicando tratamento de valores ausentes (preenchimento com mediana)")
        
        # Preencher NaNs com a mediana para colunas numéricas
        from sklearn.impute import SimpleImputer
        imputer = SimpleImputer(strategy='median')
        
        X_train_modelo = pd.DataFrame(
            imputer.fit_transform(X_train_modelo), 
            columns=X_train_modelo.columns, 
            index=X_train_modelo.index
        )
        
        X_test_modelo = pd.DataFrame(
            imputer.transform(X_test_modelo), 
            columns=X_test_modelo.columns, 
            index=X_test_modelo.index
        )
        
        print("   Valores NaN tratados com sucesso")
    
    normalizacao_status = "COM" if dataset['normalizacao_aplicada'] else "SEM"
    print(f"   Dataset: {normalizacao_status} normalização")
    
    inicio = time.time()
    modelo = config['modelo']
    modelo.fit(X_train_modelo, y_train_modelo)
    fim = time.time()
    tempo_treinamento = fim - inicio
    
    print(f"   Tempo de treinamento: {tempo_treinamento:.3f} segundos")
    
    inicio_pred = time.time()
    y_pred = modelo.predict(X_test_modelo)
    y_pred_proba = modelo.predict_proba(X_test_modelo)[:, 1]
    fim_pred = time.time()
    tempo_predicao = fim_pred - inicio_pred
    
    print(f"   Tempo de predição: {tempo_predicao:.3f} segundos")
    
    accuracy = accuracy_score(y_test_modelo, y_pred)
    precision = precision_score(y_test_modelo, y_pred)
    recall = recall_score(y_test_modelo, y_pred)
    f1 = f1_score(y_test_modelo, y_pred)
    roc_auc = roc_auc_score(y_test_modelo, y_pred_proba)
    
    print("   Executando validação cruzada...")
    # Aplicar mesmo tratamento de NaN para validação cruzada
    X_train_cv = X_train_modelo.copy()
    if X_train_cv.isnull().sum().sum() > 0:
        X_train_cv = pd.DataFrame(
            SimpleImputer(strategy='median').fit_transform(X_train_cv),
            columns=X_train_cv.columns,
            index=X_train_cv.index
        )
    
    cv_scores = cross_val_score(config['modelo'], X_train_cv, y_train_modelo, 
                               cv=5, scoring='f1', n_jobs=-1)
    cv_mean = cv_scores.mean()
    cv_std = cv_scores.std()
    
    resultados_modelos[nome_modelo] = {
        'modelo': modelo,
        'categoria': config['categoria'],
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc,
        'cv_f1_mean': cv_mean,
        'cv_f1_std': cv_std,
        'tempo_treinamento': tempo_treinamento,
        'tempo_predicao': tempo_predicao,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'y_test': y_test_modelo,
        'dataset_usado': config['dataset_key'],
        'normalizacao': dataset['normalizacao_aplicada']
    }
    
    print(f"   Métricas no teste:")
    print(f"      • Accuracy: {accuracy:.4f}")
    print(f"      • F1-Score: {f1:.4f}")
    print(f"      • ROC-AUC:  {roc_auc:.4f}")
    print(f"   Validação Cruzada F1: {cv_mean:.4f} (±{cv_std:.4f})")
    
    print(f"   {nome_modelo} treinado com sucesso")

print(f"\nTodos os {len(modelos_config)} modelos treinados com sucesso")
print("Prontos para avaliação comparativa detalhada")

print(f"\nVariáveis criadas:")
print(f"   • resultados_modelos: {len(resultados_modelos)} modelos treinados")
print(f"   • modelos_config: Configurações e justificativas")
print(f"   • datasets_por_categoria: Datasets específicos por tipo")

In [ ]:
# Etapa 4: Avaliação Comparativa dos Modelos

print("Comparação detalhada dos modelos treinados")

df_comparacao = pd.DataFrame({
    nome: {
        'Accuracy': resultado['accuracy'],
        'Precision': resultado['precision'],
        'Recall': resultado['recall'],
        'F1-Score': resultado['f1_score'],
        'ROC-AUC': resultado['roc_auc'],
        'CV F1 Mean': resultado['cv_f1_mean'],
        'CV F1 Std': resultado['cv_f1_std'],
        'Tempo Treino (s)': resultado['tempo_treinamento'],
        'Tempo Pred (s)': resultado['tempo_predicao'],
        'Normalização': 'Sim' if resultado['normalizacao'] else 'Não'
    }
    for nome, resultado in resultados_modelos.items()
}).T

print("Tabela comparativa completa:")
print(df_comparacao.round(4))

print(f"\nMelhores modelos por métrica:")

metricas_principais = ['Accuracy', 'F1-Score', 'ROC-AUC', 'CV F1 Mean']
for metrica in metricas_principais:
    melhor_modelo = df_comparacao[metrica].idxmax()
    melhor_valor = df_comparacao[metrica].max()
    print(f"   • {metrica:12s}: {melhor_modelo} ({melhor_valor:.4f})")

print(f"\nRanking geral (média das métricas normalizadas):")

metricas_para_ranking = ['Accuracy', 'F1-Score', 'ROC-AUC', 'CV F1 Mean']
df_normalizado = df_comparacao[metricas_para_ranking].copy()

for metrica in metricas_para_ranking:
    min_val = df_normalizado[metrica].min()
    max_val = df_normalizado[metrica].max()
    df_normalizado[metrica] = (df_normalizado[metrica] - min_val) / (max_val - min_val)

df_normalizado['Score_Geral'] = df_normalizado.mean(axis=1)
ranking_geral = df_normalizado.sort_values('Score_Geral', ascending=False)

for i, (modelo, row) in enumerate(ranking_geral.iterrows(), 1):
    posicao = f"{i}°" 
    print(f"{posicao} {modelo:20s} | Score: {row['Score_Geral']:.4f}")

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

metricas_viz = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']
df_viz = df_comparacao[metricas_viz]

ax = axes[0]
df_viz.plot(kind='bar', ax=ax, width=0.8)
ax.set_title('Comparação de Métricas por Modelo', fontsize=12, fontweight='bold')
ax.set_ylabel('Score')
ax.set_xlabel('Modelos')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(axis='y', alpha=0.3)
ax.tick_params(axis='x', rotation=45)

ax = axes[1]
tempos_treino = df_comparacao['Tempo Treino (s)'].sort_values()
cores_tempo = ['green' if x < 0.1 else 'orange' if x < 1 else 'red' for x in tempos_treino]
tempos_treino.plot(kind='bar', ax=ax, color=cores_tempo)
ax.set_title('Tempo de Treinamento por Modelo', fontsize=12, fontweight='bold')
ax.set_ylabel('Segundos')
ax.set_xlabel('Modelos')
ax.tick_params(axis='x', rotation=45)
ax.grid(axis='y', alpha=0.3)

ax = axes[2]
for modelo, resultado in resultados_modelos.items():
    cor = 'blue' if 'Sensíveis à Escala' in resultado['categoria'] else 'red'
    ax.scatter(resultado['accuracy'], resultado['f1_score'], 
              s=100, c=cor, alpha=0.7, label=modelo)
ax.set_xlabel('Accuracy')
ax.set_ylabel('F1-Score')
ax.set_title('Accuracy vs F1-Score', fontsize=12, fontweight='bold')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(alpha=0.3)

ax = axes[3]
modelos_nomes = list(resultados_modelos.keys())
cv_means = [resultados_modelos[m]['cv_f1_mean'] for m in modelos_nomes]
cv_stds = [resultados_modelos[m]['cv_f1_std'] for m in modelos_nomes]

bars = ax.bar(range(len(modelos_nomes)), cv_means, yerr=cv_stds, 
              capsize=5, alpha=0.7, color=['blue', 'blue', 'red', 'red'])
ax.set_xticks(range(len(modelos_nomes)))
ax.set_xticklabels([nome[:15] + '...' if len(nome) > 15 else nome for nome in modelos_nomes], rotation=45)
ax.set_ylabel('F1-Score')
ax.set_title('Validação Cruzada (5-fold) - F1-Score', fontsize=12, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

melhor_modelo_nome = ranking_geral.index[0]
melhor_resultado = resultados_modelos[melhor_modelo_nome]

cm = confusion_matrix(melhor_resultado['y_test'], melhor_resultado['y_pred'])
ax = axes[4]
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=['Não Cancelou', 'Cancelou'],
            yticklabels=['Não Cancelou', 'Cancelou'])
ax.set_title(f'Matriz de Confusão\n{melhor_modelo_nome}', fontsize=12, fontweight='bold')
ax.set_xlabel('Predição')
ax.set_ylabel('Real')

ax = axes[5]
colors = ['blue', 'green', 'red', 'orange']
for i, (nome_modelo, resultado) in enumerate(resultados_modelos.items()):
    fpr, tpr, _ = roc_curve(resultado['y_test'], resultado['y_pred_proba'])
    auc_score = auc(fpr, tpr)
    ax.plot(fpr, tpr, color=colors[i], lw=2, 
            label=f'{nome_modelo[:15]}... (AUC={auc_score:.3f})')

ax.plot([0, 1], [0, 1], 'k--', lw=2, label='Random (AUC=0.5)')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('Taxa de Falsos Positivos')
ax.set_ylabel('Taxa de Verdadeiros Positivos')
ax.set_title('Curvas ROC - Comparação', fontsize=12, fontweight='bold')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nAnálise por categoria de modelo:")

categorias_performance = {}
for nome_modelo, resultado in resultados_modelos.items():
    categoria = resultado['categoria']
    if categoria not in categorias_performance:
        categorias_performance[categoria] = []
    categorias_performance[categoria].append({
        'nome': nome_modelo,
        'f1': resultado['f1_score'],
        'auc': resultado['roc_auc'],
        'tempo': resultado['tempo_treinamento']
    })

for categoria, modelos in categorias_performance.items():
    print(f"\n{categoria.upper()}:")
    f1_medio = np.mean([m['f1'] for m in modelos])
    auc_medio = np.mean([m['auc'] for m in modelos])
    tempo_medio = np.mean([m['tempo'] for m in modelos])
    
    print(f"   Performance média: F1={f1_medio:.4f}, AUC={auc_medio:.4f}")
    print(f"   Tempo médio de treino: {tempo_medio:.4f}s")
    print(f"   Melhor modelo: {max(modelos, key=lambda x: x['f1'])['nome']}")

print(f"\nRecomendação final:")

melhor_geral = ranking_geral.index[0]
melhor_resultado_final = resultados_modelos[melhor_geral]

print(f"MODELO RECOMENDADO: {melhor_geral}")
print(f"   F1-Score: {melhor_resultado_final['f1_score']:.4f}")
print(f"   ROC-AUC: {melhor_resultado_final['roc_auc']:.4f}")
print(f"   Tempo treino: {melhor_resultado_final['tempo_treinamento']:.4f}s")
print(f"   Normalização: {'Sim' if melhor_resultado_final['normalizacao'] else 'Não'}")

print(f"\nAvaliação comparativa concluída")
print("Modelo recomendado selecionado para implementação")

In [ ]:
# Etapa 5: Síntese da Modelagem Preditiva

print("Síntese completa da modelagem preditiva")

print("Estratégia de modelagem implementada:")

print("1. DIVISÃO DOS DADOS:")
print(f"     • Estratégia: {divisao_escolhida} (treino/teste)")
print("     • Método: Divisão estratificada para manter proporção das classes")
print("     • Validação: Verificação da distribuição em treino e teste")

print("\n2. PREPARAÇÃO ESPECÍFICA POR MODELO:")
print("     • Modelos sensíveis à escala: Normalização com StandardScaler")
print("     • Modelos baseados em árvore: Dados na escala original")
print("     • Justificativa: Otimização específica para cada tipo de algoritmo")

print("\n3. SELEÇÃO E TREINAMENTO DOS MODELOS:")
algoritmos_implementados = list(modelos_config.keys())
print(f"     • Total de algoritmos: {len(algoritmos_implementados)}")
for algo in algoritmos_implementados:
    print(f"       - {algo}")

print("\n4. VALIDAÇÃO E AVALIAÇÃO:")
print("     • Métricas: Accuracy, Precision, Recall, F1-Score, ROC-AUC")
print("     • Validação cruzada: 5-fold stratified")
print("     • Comparação: Ranking baseado em métricas normalizadas")

melhor_modelo_final = ranking_geral.index[0]
resultado_melhor = resultados_modelos[melhor_modelo_final]

print(f"\n5. RESULTADO FINAL:")
print(f"     • Modelo vencedor: {melhor_modelo_final}")
print(f"     • F1-Score: {resultado_melhor['f1_score']:.4f}")
print(f"     • ROC-AUC: {resultado_melhor['roc_auc']:.4f}")
print(f"     • Accuracy: {resultado_melhor['accuracy']:.4f}")

estrategia_implementacao = {
    'modelo_final': melhor_modelo_final,
    'metricas_finais': {
        'f1_score': resultado_melhor['f1_score'],
        'roc_auc': resultado_melhor['roc_auc'],
        'accuracy': resultado_melhor['accuracy'],
        'precision': resultado_melhor['precision'],
        'recall': resultado_melhor['recall']
    },
    'preprocessing_necessario': 'StandardScaler' if resultado_melhor['normalizacao'] else 'Sem normalização',
    'tempo_treinamento': resultado_melhor['tempo_treinamento'],
    'validacao_cruzada': {
        'cv_f1_mean': resultado_melhor['cv_f1_mean'],
        'cv_f1_std': resultado_melhor['cv_f1_std']
    }
}

print(f"\n6. INFORMAÇÕES PARA IMPLEMENTAÇÃO:")
print(f"     • Pré-processamento: {estrategia_implementacao['preprocessing_necessario']}")
print(f"     • Tempo de treinamento: {estrategia_implementacao['tempo_treinamento']:.4f}s")
print(f"     • Validação cruzada F1: {estrategia_implementacao['validacao_cruzada']['cv_f1_mean']:.4f} ± {estrategia_implementacao['validacao_cruzada']['cv_f1_std']:.4f}")

print(f"\nSíntese da modelagem preditiva concluída")
print(f"Variável 'estrategia_implementacao' criada com todos os detalhes")

## 4. Modelagem Preditiva

### Estratégia de Modelagem:

**Modelos Selecionados:**
- **Regressão Logística** - Linear, interpretável
- **K-Nearest Neighbors** - Não-paramétrico, fronteiras complexas  
- **Árvore de Decisão** - Regras interpretáveis
- **Random Forest** - Ensemble de árvores, robusto

**Justificativas Técnicas:**
- **Normalização necessária** para modelos baseados em distância/gradiente
- **Sem normalização** para modelos baseados em árvore (decisões por limites)
- **Comparação justa** entre diferentes abordagens algorítmicas
- **Aproveitamento** das análises de correlação e seleção de features

In [ ]:
# Análise de Importância das Variáveis por Modelo

print("ANÁLISE DE IMPORTÂNCIA DAS VARIÁVEIS")
print("=" * 50)

# 1. Regressão Logística - Análise de Coeficientes
if 'Regressão Logística' in resultados_modelos:
    print("\n1. REGRESSÃO LOGÍSTICA - Análise de Coeficientes")
    print("-" * 50)
    
    modelo_lr = resultados_modelos['Regressão Logística']['modelo']
    
    # Obter coeficientes
    coeficientes = modelo_lr.coef_[0]
    
    # Obter nomes das features do dataset normalizado
    dataset_lr = datasets_por_categoria['Sensíveis à Escala']
    feature_names = dataset_lr['X_train'].columns
    
    # Criar dataframe com coeficientes
    df_coef = pd.DataFrame({
        'Variável': feature_names,
        'Coeficiente': coeficientes,
        'Coef_Absoluto': np.abs(coeficientes),
        'Odds_Ratio': np.exp(coeficientes),
        'Impacto': ['Aumenta Churn' if c > 0 else 'Reduz Churn' for c in coeficientes]
    })
    
    # Ordenar por importância (valor absoluto)
    df_coef = df_coef.sort_values('Coef_Absoluto', ascending=False)
    
    print("Top 10 variáveis mais impactantes:")
    for i, row in df_coef.head(10).iterrows():
        print(f"{row.name + 1:2d}. {row['Variável']:25s} | Coef: {row['Coeficiente']:8.4f} | OR: {row['Odds_Ratio']:6.3f} | {row['Impacto']}")
    
    # Interpretação dos odds ratios
    print("\nInterpretação dos Odds Ratios:")
    for i, row in df_coef.head(5).iterrows():
        if row['Odds_Ratio'] > 1:
            aumento = (row['Odds_Ratio'] - 1) * 100
            print(f"- {row['Variável']}: Aumenta chance de churn em {aumento:.1f}%")
        else:
            reducao = (1 - row['Odds_Ratio']) * 100
            print(f"- {row['Variável']}: Reduz chance de churn em {reducao:.1f}%")
    
    # Visualização
    plt.figure(figsize=(12, 6))
    
    # Gráfico dos coeficientes
    plt.subplot(1, 2, 1)
    top_10_coef = df_coef.head(10)
    colors = ['red' if x < 0 else 'blue' for x in top_10_coef['Coeficiente']]
    plt.barh(range(len(top_10_coef)), top_10_coef['Coeficiente'], color=colors, alpha=0.7)
    plt.yticks(range(len(top_10_coef)), [name[:20] for name in top_10_coef['Variável']])
    plt.xlabel('Coeficiente')
    plt.title('Top 10 Coeficientes - Regressão Logística')
    plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
    plt.grid(axis='x', alpha=0.3)
    
    # Gráfico dos odds ratios
    plt.subplot(1, 2, 2)
    plt.barh(range(len(top_10_coef)), top_10_coef['Odds_Ratio'], 
             color=['green' if x < 1 else 'red' for x in top_10_coef['Odds_Ratio']], alpha=0.7)
    plt.yticks(range(len(top_10_coef)), [name[:20] for name in top_10_coef['Variável']])
    plt.xlabel('Odds Ratio')
    plt.title('Top 10 Odds Ratios - Regressão Logística')
    plt.axvline(x=1, color='black', linestyle='-', alpha=0.3)
    plt.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Armazenar para relatório final
    lr_insights = {
        'top_5_vars': df_coef.head(5)['Variável'].tolist(),
        'strongest_positive': df_coef[df_coef['Coeficiente'] > 0].iloc[0]['Variável'],
        'strongest_negative': df_coef[df_coef['Coeficiente'] < 0].iloc[0]['Variável'] if len(df_coef[df_coef['Coeficiente'] < 0]) > 0 else None
    }

In [ ]:
# 2. Random Forest - Análise de Importância das Variáveis
if 'Random Forest' in resultados_modelos:
    print("\n2. RANDOM FOREST - Importância das Variáveis")
    print("-" * 50)
    
    modelo_rf = resultados_modelos['Random Forest']['modelo']
    
    importancias = modelo_rf.feature_importances_
    
    dataset_rf = datasets_por_categoria['Baseados em Árvore']
    feature_names_rf = dataset_rf['X_train'].columns
    
    df_importancia = pd.DataFrame({
        'Variável': feature_names_rf,
        'Importância': importancias,
        'Importância_Pct': importancias * 100
    })
    
    df_importancia = df_importancia.sort_values('Importância', ascending=False)
    
    print("Top 10 variáveis mais importantes:")
    for i, row in df_importancia.head(10).iterrows():
        print(f"{i+1:2d}. {row['Variável']:25s} | Importância: {row['Importância']:6.4f} ({row['Importância_Pct']:5.1f}%)")
    
    # Calcular importância cumulativa
    df_importancia['Importância_Cumulativa'] = df_importancia['Importância'].cumsum()
    
    print(f"\nImportância cumulativa:")
    print(f"Top 5 variáveis: {df_importancia.head(5)['Importância_Cumulativa'].iloc[-1]:.1%}")
    print(f"Top 10 variáveis: {df_importancia.head(10)['Importância_Cumulativa'].iloc[-1]:.1%}")
    
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    top_10_imp = df_importancia.head(10)
    plt.barh(range(len(top_10_imp)), top_10_imp['Importância'], color='forestgreen', alpha=0.7)
    plt.yticks(range(len(top_10_imp)), [name[:20] for name in top_10_imp['Variável']])
    plt.xlabel('Importância')
    plt.title('Top 10 Importância - Random Forest')
    plt.grid(axis='x', alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.plot(range(1, len(df_importancia.head(15)) + 1), 
             df_importancia.head(15)['Importância_Cumulativa'], 'o-', color='darkgreen')
    plt.xlabel('Número de Variáveis')
    plt.ylabel('Importância Cumulativa')
    plt.title('Importância Cumulativa - Top 15')
    plt.grid(alpha=0.3)
    plt.axhline(y=0.8, color='red', linestyle='--', alpha=0.7, label='80%')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Armazenar para relatório final
    rf_insights = {
        'top_5_vars': df_importancia.head(5)['Variável'].tolist(),
        'most_important': df_importancia.iloc[0]['Variável'],
        'top_5_cumulative': df_importancia.head(5)['Importância_Cumulativa'].iloc[-1]
    }

In [ ]:
# 3. K-Nearest Neighbors - Análise por Proximidade
if 'K-Nearest Neighbors' in resultados_modelos:
    print("\n3. K-NEAREST NEIGHBORS - Análise por Proximidade")
    print("-" * 50)
    
    modelo_knn = resultados_modelos['K-Nearest Neighbors']['modelo']
    dataset_knn = datasets_por_categoria['Sensíveis à Escala']
    X_test_knn = dataset_knn['X_test']
    y_test_knn = dataset_knn['y_test']
    
    print("Análise de influência das variáveis baseada em distâncias:")

    from sklearn.metrics.pairwise import euclidean_distances

    X_train_knn = dataset_knn['X_train']
    y_train_knn = dataset_knn['y_train']
    
    X_churn = X_train_knn[y_train_knn == 1]
    X_no_churn = X_train_knn[y_train_knn == 0]
    
    # Calcular variâncias das features (maior variância = maior impacto na distância)
    feature_variance = X_train_knn.var()
    feature_std = X_train_knn.std()
    
    # análise de relevância baseada na variabilidade
    df_knn_relevance = pd.DataFrame({
        'Variável': X_train_knn.columns,
        'Variância': feature_variance,
        'Desvio_Padrão': feature_std,
        'Relevância_Normalizada': (feature_std - feature_std.min()) / (feature_std.max() - feature_std.min())
    })
    
    df_knn_relevance = df_knn_relevance.sort_values('Relevância_Normalizada', ascending=False)
    
    print("Top 10 variáveis mais relevantes para cálculo de distância:")
    for i, row in df_knn_relevance.head(10).iterrows():
        print(f"{i+1:2d}. {row['Variável']:25s} | Std: {row['Desvio_Padrão']:6.4f} | Relevância: {row['Relevância_Normalizada']:6.4f}")
    
    # relatório final
    knn_insights = {
        'top_5_vars': df_knn_relevance.head(5)['Variável'].tolist(),
        'most_relevant': df_knn_relevance.iloc[0]['Variável']
    }

# 4. Árvore de Decisão - Análise de Importância
if 'Árvore de Decisão' in resultados_modelos:
    print("\n4. ÁRVORE DE DECISÃO - Importância das Variáveis")
    print("-" * 50)
    
    modelo_dt = resultados_modelos['Árvore de Decisão']['modelo']
    
    importancias_dt = modelo_dt.feature_importances_
    
    #  nomes das features
    dataset_dt = datasets_por_categoria['Baseados em Árvore']
    feature_names_dt = dataset_dt['X_train'].columns
    
    # dataframe com importâncias
    df_dt_importancia = pd.DataFrame({
        'Variável': feature_names_dt,
        'Importância': importancias_dt,
        'Importância_Pct': importancias_dt * 100
    })
    
    #por importância
    df_dt_importancia = df_dt_importancia.sort_values('Importância', ascending=False)
    
    print("Top 10 variáveis mais importantes para divisões:")
    for i, row in df_dt_importancia.head(10).iterrows():
        print(f"{i+1:2d}. {row['Variável']:25s} | Importância: {row['Importância']:6.4f} ({row['Importância_Pct']:5.1f}%)")

    from sklearn import tree
    
    print("\nPrincipais regras de decisão (primeiros níveis):")
    tree_rules = tree.export_text(modelo_dt, feature_names=list(feature_names_dt), max_depth=3)
    print(tree_rules[:500] + "...")
    
    #  relatório final
    dt_insights = {
        'top_5_vars': df_dt_importancia.head(5)['Variável'].tolist(),
        'most_important': df_dt_importancia.iloc[0]['Variável'],
        'top_3_importance': df_dt_importancia.head(3)['Importância'].sum()
    }

In [ ]:
# Consolidação das Análises de Importância
print("\n" + "="*60)
print("CONSOLIDAÇÃO DAS ANÁLISES DE IMPORTÂNCIA")
print("="*60)

# Criar dicionário consolidado
all_insights = {}
if 'lr_insights' in locals():
    all_insights['Regressão Logística'] = lr_insights
if 'rf_insights' in locals():
    all_insights['Random Forest'] = rf_insights  
if 'knn_insights' in locals():
    all_insights['K-Nearest Neighbors'] = knn_insights
if 'dt_insights' in locals():
    all_insights['Árvore de Decisão'] = dt_insights

print("\nTOP 5 VARIÁVEIS MAIS IMPORTANTES POR MODELO:")
print("-" * 50)

for modelo, insights in all_insights.items():
    print(f"\n{modelo}:")
    for i, var in enumerate(insights['top_5_vars'], 1):
        print(f"  {i}. {var}")

# Análise de consenso entre modelos
all_important_vars = []
for insights in all_insights.values():
    all_important_vars.extend(insights['top_5_vars'])

# Contar frequência de cada variável
from collections import Counter
var_frequency = Counter(all_important_vars)

print("\nCONSENSO ENTRE MODELOS - Variáveis mais citadas:")
print("-" * 50)
for var, count in var_frequency.most_common(10):
    models_citing = [modelo for modelo, insights in all_insights.items() if var in insights['top_5_vars']]
    print(f"{var:30s} | Citada por {count} modelo(s): {', '.join(models_citing)}")

# Visualização comparativa
if len(all_insights) > 0:
    plt.figure(figsize=(15, 8))
    
    # Criar matriz de importância por modelo
    top_vars_consensus = [var for var, count in var_frequency.most_common(10)]
    
    # Matriz de presença (1 se está no top 5, 0 caso contrário)
    matrix_data = []
    model_names = list(all_insights.keys())
    
    for var in top_vars_consensus:
        row = []
        for modelo in model_names:
            if var in all_insights[modelo]['top_5_vars']:
                # Posição na lista (1 = mais importante)
                pos = all_insights[modelo]['top_5_vars'].index(var) + 1
                row.append(6 - pos)  # Inverter para que 5 = mais importante
            else:
                row.append(0)
        matrix_data.append(row)
    
    # Heatmap
    plt.subplot(1, 2, 1)
    sns.heatmap(matrix_data, 
                xticklabels=model_names,
                yticklabels=[var[:25] for var in top_vars_consensus],
                annot=True, 
                cmap='YlOrRd', 
                cbar_kws={'label': 'Importância (5=máxima)'})
    plt.title('Consenso de Importância entre Modelos')
    plt.xlabel('Modelos')
    plt.ylabel('Variáveis')
    
    # Gráfico de frequência
    plt.subplot(1, 2, 2)
    vars_plot = [var[:20] for var, _ in var_frequency.most_common(8)]
    counts_plot = [count for _, count in var_frequency.most_common(8)]
    
    plt.barh(range(len(vars_plot)), counts_plot, color='steelblue', alpha=0.7)
    plt.yticks(range(len(vars_plot)), vars_plot)
    plt.xlabel('Número de Modelos que Citam')
    plt.title('Frequência de Citação entre Modelos')
    plt.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Identificar variáveis de consenso (citadas por pelo menos 50% dos modelos)
consensus_threshold = len(all_insights) * 0.5
consensus_vars = [var for var, count in var_frequency.items() if count >= consensus_threshold]

print(f"\nVARIÁVEIS DE CONSENSO (citadas por >= {consensus_threshold:.0f} modelos):")
print("-" * 55)
for var in consensus_vars:
    count = var_frequency[var]
    print(f"- {var} (citada por {count} modelo(s))")

# Armazenar resultados para relatório final
consensus_analysis = {
    'top_consensus_vars': consensus_vars[:5] if len(consensus_vars) >= 5 else consensus_vars,
    'var_frequency': dict(var_frequency.most_common(10)),
    'models_analyzed': list(all_insights.keys())
}

print(f"\nAnálise de importância concluída para {len(all_insights)} modelos.")

## 5. Relatório Final e Estratégias de Retenção

### Análise Detalhada dos Fatores de Evasão

In [ ]:
# RELATÓRIO DETALHADO - FATORES DE EVASÃO E ESTRATÉGIAS DE RETENÇÃO

print("RELATÓRIO FINAL - ANÁLISE DE EVASÃO TELECOM X")
print("="*60)

# Recuperar informações dos modelos
melhor_modelo = ranking_geral.index[0] if 'ranking_geral' in locals() else "Não definido"
melhor_performance = resultados_modelos[melhor_modelo] if melhor_modelo in resultados_modelos else {}

print(f"\nRESUMO EXECUTIVO")
print("-" * 20)
print(f"• Dataset analisado: {len(y_modelagem):,} clientes" if 'y_modelagem' in locals() else "• Dataset: Não carregado")
if 'distribuicao_original' in locals():
    churn_rate = distribuicao_original[1] / len(y_modelagem) * 100
    print(f"• Taxa de evasão atual: {churn_rate:.1f}%")
print(f"• Modelos avaliados: {len(resultados_modelos)}")
if melhor_performance:
    print(f"• Melhor modelo: {melhor_modelo}")
    print(f"• F1-Score alcançado: {melhor_performance['f1_score']:.4f}")
    print(f"• ROC-AUC alcançado: {melhor_performance['roc_auc']:.4f}")

print(f"\n1. PRINCIPAIS FATORES QUE INFLUENCIAM A EVASÃO")
print("-" * 50)

if 'consensus_vars' in locals() and consensus_vars:
    print("Baseado na análise de consenso entre modelos:")
    for i, var in enumerate(consensus_vars[:5], 1):
        models_citing = [modelo for modelo, insights in all_insights.items() if var in insights.get('top_5_vars', [])]
        print(f"  {i}. {var}")
        print(f"     - Identificada por: {', '.join(models_citing)}")
        
        # Adicionar interpretação específica por tipo de variável
        if any(term in var.lower() for term in ['tenure', 'tempo']):
            print("     - Interpretação: Tempo de relacionamento com a empresa")
        elif any(term in var.lower() for term in ['charges', 'total', 'monthly']):
            print("     - Interpretação: Aspecto financeiro do serviço")
        elif any(term in var.lower() for term in ['contract', 'contrato']):
            print("     - Interpretação: Tipo de vínculo contratual")
        elif any(term in var.lower() for term in ['service', 'internet', 'phone']):
            print("     - Interpretação: Tipo de serviço utilizado")
        else:
            print("     - Interpretação: Fator comportamental/demográfico")
else:
    print("Execute as análises de importância primeiro.")

print(f"\n2. ANÁLISE POR MODELO")
print("-" * 25)

for modelo, insights in all_insights.items():
    print(f"\n{modelo}:")
    performance = resultados_modelos[modelo]
    print(f"  Performance: F1={performance['f1_score']:.3f}, AUC={performance['roc_auc']:.3f}")
    print(f"  Principal fator identificado: {insights.get('most_important', insights.get('top_5_vars', ['N/A'])[0])}")
    
    if modelo == 'Regressão Logística':
        print("  Metodologia: Análise de coeficientes e odds ratios")
        print("  Vantagem: Interpretabilidade direta do impacto")
    elif modelo == 'Random Forest':
        print("  Metodologia: Importância baseada em redução de impureza")
        print("  Vantagem: Captura interações complexas entre variáveis")
    elif modelo == 'K-Nearest Neighbors':
        print("  Metodologia: Relevância baseada em variabilidade para distância")
        print("  Vantagem: Identifica padrões locais nos dados")
    elif modelo == 'Árvore de Decisão':
        print("  Metodologia: Importância baseada em divisões da árvore")
        print("  Vantagem: Regras de decisão explícitas")

print(f"\n3. ESTRATÉGIAS DE RETENÇÃO BASEADAS NOS RESULTADOS")
print("-" * 50)

# Estratégias baseadas nos principais fatores identificados
strategies = []

if 'consensus_vars' in locals():
    for var in consensus_vars[:3]:
        if any(term in var.lower() for term in ['tenure', 'tempo']):
            strategies.append({
                'fator': var,
                'estrategia': 'Programa de Fidelidade Progressiva',
                'acao': 'Criar benefícios crescentes baseados no tempo de relacionamento',
                'implementacao': [
                    'Descontos progressivos por tempo de permanência',
                    'Upgrades gratuitos após períodos específicos',
                    'Atendimento prioritário para clientes antigos'
                ]
            })
        elif any(term in var.lower() for term in ['charges', 'total', 'monthly']):
            strategies.append({
                'fator': var,
                'estrategia': 'Otimização de Preços e Pacotes',
                'acao': 'Ajustar estrutura de preços para diferentes perfis',
                'implementacao': [
                    'Análise de sensibilidade ao preço por segmento',
                    'Pacotes personalizados baseados no uso',
                    'Promoções direcionadas para clientes em risco'
                ]
            })
        elif any(term in var.lower() for term in ['contract', 'contrato']):
            strategies.append({
                'fator': var,
                'estrategia': 'Flexibilização Contratual',
                'acao': 'Oferecer opções contratuais mais atrativas',
                'implementacao': [
                    'Contratos com flexibilidade de mudança',
                    'Períodos de teste sem compromisso',
                    'Migração facilitada entre planos'
                ]
            })

# Imprimir estratégias
for i, strategy in enumerate(strategies, 1):
    print(f"\nEstratégia {i}: {strategy['estrategia']}")
    print(f"  Fator relacionado: {strategy['fator']}")
    print(f"  Objetivo: {strategy['acao']}")
    print("  Implementações sugeridas:")
    for impl in strategy['implementacao']:
        print(f"    - {impl}")

print(f"\n4. IMPLEMENTAÇÃO DO SISTEMA PREDITIVO")
print("-" * 40)

print("Recomendações para operacionalização:")
print("\nA. Scoring de Clientes")
print("  - Aplicar modelo preditivo mensalmente")
print("  - Classificar clientes por probabilidade de churn")
print("  - Criar alertas automáticos para casos de alto risco")

print("\nB. Segmentação para Ações")
if melhor_performance:
    threshold_high = 0.7
    threshold_medium = 0.4
    print(f"  - Alto risco (p > {threshold_high}): Intervenção imediata")
    print(f"  - Médio risco ({threshold_medium} < p < {threshold_high}): Acompanhamento próximo") 
    print(f"  - Baixo risco (p < {threshold_medium}): Manutenção regular")

print("\nC. Monitoramento e Ajustes")
print("  - Avaliar efetividade das estratégias implementadas")
print("  - Retreinar modelo com novos dados trimestralmente")
print("  - Acompanhar mudanças nos fatores de risco")

print(f"\n5. IMPACTO POTENCIAL")
print("-" * 20)

if 'churn_rate' in locals() and melhor_performance:
    current_churn = churn_rate / 100
    model_precision = melhor_performance.get('precision', 0.5)
    model_recall = melhor_performance.get('recall', 0.5)
    
    print(f"Cenário atual:")
    print(f"  - Taxa de churn: {churn_rate:.1f}%")
    print(f"  - Precisão do modelo: {model_precision:.1%}")
    print(f"  - Recall do modelo: {model_recall:.1%}")
    
    potential_reduction = 0.3  # Assumindo 30% de efetividade das ações de retenção
    print(f"\nCenário otimista (30% de efetividade nas ações):")
    new_churn_rate = current_churn * (1 - model_recall * potential_reduction)
    reduction = (current_churn - new_churn_rate) / current_churn * 100
    print(f"  - Nova taxa de churn estimada: {new_churn_rate*100:.1f}%")
    print(f"  - Redução potencial: {reduction:.1f}%")

print(f"\n6. PRÓXIMOS PASSOS")
print("-" * 18)
print("1. Validar modelo em ambiente de produção")
print("2. Implementar sistema de scoring automático")  
print("3. Desenvolver campanhas específicas por segmento")
print("4. Criar dashboards de monitoramento")
print("5. Treinar equipes comerciais com insights dos modelos")
print("6. Estabelecer métricas de sucesso e ROI")

print("\n" + "="*60)
print("RELATÓRIO CONCLUÍDO")
print("="*60)

## 2.5. Análise de Correlação e Seleção de Variáveis

### Objetivos desta Análise:
1. **Matriz de Correlação**: Identificar relações lineares entre variáveis numéricas
2. **Correlação com Target**: Descobrir quais variáveis têm maior correlação com churn
3. **Análises Específicas**:
   - Tempo de contrato × Evasão
   - Total gasto × Evasão
   - Outros padrões relevantes
4. **Visualizações Interativas**: Boxplots e scatter plots para insights visuais
5. **Seleção de Features**: Identificar variáveis mais relevantes para modelagem

In [ ]:
# Análise 1: Matriz de Correlação Geral

print("Análise de correlação - variáveis numéricas")

try:
    dados_para_correlacao = pd.concat([X_final, pd.Series(y_encoded, name='Churn')], axis=1)
    print("Usando dados pré-processados para análise de correlação")
except:
    dados_para_correlacao = dados.copy()
    print("Usando dados originais - execute as células de pré-processamento primeiro")

dados_numericos = dados_para_correlacao.select_dtypes(include=[np.number])

print(f"\nVariáveis numéricas identificadas: {len(dados_numericos.columns)}")
for i, col in enumerate(dados_numericos.columns):
    print(f"{i+1:2d}. {col}")

matriz_correlacao = dados_numericos.corr()

plt.figure(figsize=(14, 10))

mask = np.triu(np.ones_like(matriz_correlacao, dtype=bool))
sns.heatmap(matriz_correlacao, 
            mask=mask,
            annot=True, 
            fmt='.3f', 
            center=0,
            cmap='RdBu_r', 
            square=True,
            cbar_kws={'label': 'Coeficiente de Correlação'})

plt.title('Matriz de Correlação - Variáveis Numéricas', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print(f"\nEstatísticas da matriz de correlação:")
print(f"   • Dimensões: {matriz_correlacao.shape}")
print(f"   • Correlação máxima (excl. diagonal): {matriz_correlacao.where(~np.eye(len(matriz_correlacao), dtype=bool)).max().max():.3f}")
print(f"   • Correlação mínima: {matriz_correlacao.where(~np.eye(len(matriz_correlacao), dtype=bool)).min().min():.3f}")

correlacoes_fortes = []
for i in range(len(matriz_correlacao.columns)):
    for j in range(i+1, len(matriz_correlacao.columns)):
        corr_val = matriz_correlacao.iloc[i, j]
        if abs(corr_val) > 0.5:
            correlacoes_fortes.append({
                'var1': matriz_correlacao.columns[i],
                'var2': matriz_correlacao.columns[j],
                'correlacao': corr_val
            })

if correlacoes_fortes:
    print(f"\nCorrelações fortes (|r| > 0.5):")
    correlacoes_fortes_df = pd.DataFrame(correlacoes_fortes).sort_values('correlacao', key=abs, ascending=False)
    for _, row in correlacoes_fortes_df.iterrows():
        intensidade = "MUITO FORTE" if abs(row['correlacao']) > 0.8 else "FORTE" if abs(row['correlacao']) > 0.7 else "MODERADA"
        print(f"   {intensidade}: {row['var1']} ↔ {row['var2']}: {row['correlacao']:+.3f}")
else:
    print(f"\nNenhuma correlação muito forte encontrada entre variáveis")

print(f"\nAnálise de correlação geral concluída")

In [ ]:
# Análise 2: Correlação com Variável Target (Churn)

print("Correlação das variáveis com churn (evasão)")

coluna_churn = None
for col in dados_numericos.columns:
    if 'churn' in col.lower() or 'evasao' in col.lower():
        coluna_churn = col
        break

if coluna_churn is None:
    print("Coluna de churn não encontrada automaticamente")
    if 'Churn' in dados_numericos.columns:
        coluna_churn = 'Churn'
    else:
        coluna_churn = dados_numericos.columns[-1]
    print(f"Assumindo '{coluna_churn}' como variável target")

print(f"Variável target identificada: '{coluna_churn}'")

correlacoes_churn = dados_numericos.corr()[coluna_churn].drop(coluna_churn).sort_values(key=abs, ascending=False)

print(f"\nCorrelações com {coluna_churn.upper()}:")

for i, (variavel, correlacao) in enumerate(correlacoes_churn.items()):
    if abs(correlacao) > 0.7:
        intensidade = "MUITO FORTE"
    elif abs(correlacao) > 0.5:
        intensidade = "FORTE"
    elif abs(correlacao) > 0.3:
        intensidade = "MODERADA"
    elif abs(correlacao) > 0.1:
        intensidade = "FRACA"
    else:
        intensidade = "MUITO FRACA"
    
    direcao = "POSITIVA" if correlacao > 0 else "NEGATIVA"
    
    print(f"{i+1:2d}. {variavel:25s} | {correlacao:+.4f} | {intensidade} {direcao}")

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

correlacoes_churn_abs = correlacoes_churn.reindex(correlacoes_churn.abs().sort_values(ascending=True).index)
colors = ['red' if x < 0 else 'blue' for x in correlacoes_churn_abs.values]

axes[0].barh(range(len(correlacoes_churn_abs)), correlacoes_churn_abs.values, color=colors, alpha=0.7)
axes[0].set_yticks(range(len(correlacoes_churn_abs)))
axes[0].set_yticklabels(correlacoes_churn_abs.index, fontsize=10)
axes[0].set_xlabel('Coeficiente de Correlação')
axes[0].set_title(f'Correlação das Variáveis com {coluna_churn}', fontsize=12, fontweight='bold')
axes[0].axvline(x=0, color='black', linestyle='-', alpha=0.3)
axes[0].grid(axis='x', alpha=0.3)

correlacoes_abs_top10 = correlacoes_churn.abs().sort_values(ascending=False).head(10)
axes[1].bar(range(len(correlacoes_abs_top10)), correlacoes_abs_top10.values, 
           color='coral', alpha=0.7)
axes[1].set_xticks(range(len(correlacoes_abs_top10)))
axes[1].set_xticklabels(correlacoes_abs_top10.index, rotation=45, ha='right')
axes[1].set_ylabel('Correlação Absoluta')
axes[1].set_title(f'Top 10 Variáveis - Correlação Absoluta com {coluna_churn}', fontsize=12, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

top_features_correlacao = correlacoes_churn.abs().sort_values(ascending=False).head(10)

print(f"\nTop 10 variáveis mais correlacionadas com {coluna_churn.upper()}:")
for i, (var, corr_abs) in enumerate(top_features_correlacao.items()):
    corr_original = correlacoes_churn[var]
    print(f"{i+1:2d}. {var:25s} | Correlação: {corr_original:+.4f} | |r|: {corr_abs:.4f}")

print(f"\nInsights para modelagem:")
features_relevantes = top_features_correlacao[top_features_correlacao > 0.1]
print(f"   • {len(features_relevantes)} variáveis têm correlação relevante (|r| > 0.1) com {coluna_churn}")
features_fortes = top_features_correlacao[top_features_correlacao > 0.3]
print(f"   • {len(features_fortes)} variáveis têm correlação moderada/forte (|r| > 0.3) com {coluna_churn}")

if len(features_fortes) > 0:
    print(f"   • Features principais: {list(features_fortes.index)}")

print(f"\nAnálise de correlação com target concluída")

In [ ]:
### ⏰ Análise 3: Tempo de Contrato × Evasão

print("⏰ ANÁLISE: TEMPO DE CONTRATO × EVASÃO")
print("=" * 45)

# Identificar coluna relacionada ao tempo de contrato
coluna_tempo = None
possíveis_colunas_tempo = ['tenure', 'tempo', 'contract', 'tempo_contrato', 'customer.tenure']

for col in dados_para_correlacao.columns:
    col_lower = col.lower()
    if any(termo in col_lower for termo in ['tenure', 'tempo', 'contract']):
        coluna_tempo = col
        break

if coluna_tempo is None:
    print("⚠️ Coluna de tempo de contrato não encontrada automaticamente")
    print("📋 Colunas disponíveis:")
    for i, col in enumerate(dados_para_correlacao.columns):
        print(f"  {i+1:2d}. {col}")
    print("💡 Assumindo que existe uma variável relacionada ao tempo...")
    # Tentar encontrar qualquer coluna que possa representar tempo
    for col in dados_para_correlacao.columns:
        if dados_para_correlacao[col].dtype in ['int64', 'float64']:
            # Verificar se os valores fazem sentido para tempo (ex: 0-100 meses)
            if dados_para_correlacao[col].min() >= 0 and dados_para_correlacao[col].max() <= 200:
                coluna_tempo = col
                break
    
    if coluna_tempo:
        print(f"🔍 Assumindo '{coluna_tempo}' como variável de tempo")
    else:
        print("❌ Não foi possível identificar variável de tempo")

if coluna_tempo and coluna_churn:
    print(f"\n📊 ANALISANDO: {coluna_tempo} × {coluna_churn}")
    
    # Estatísticas básicas por grupo de churn
    stats_tempo = dados_para_correlacao.groupby(coluna_churn)[coluna_tempo].agg([
        'count', 'mean', 'median', 'std', 'min', 'max'
    ]).round(2)
    
    print(f"\n📈 ESTATÍSTICAS DE {coluna_tempo.upper()} POR GRUPO DE CHURN:")
    print("-" * 55)
    print(stats_tempo)
    
    # Calcular correlação específica
    correlacao_tempo_churn = dados_para_correlacao[coluna_tempo].corr(dados_para_correlacao[coluna_churn])
    print(f"\n🔗 CORRELAÇÃO {coluna_tempo} × {coluna_churn}: {correlacao_tempo_churn:+.4f}")
    
    # Interpretação da correlação
    if abs(correlacao_tempo_churn) > 0.5:
        interpretacao = "FORTE"
    elif abs(correlacao_tempo_churn) > 0.3:
        interpretacao = "MODERADA"
    elif abs(correlacao_tempo_churn) > 0.1:
        interpretacao = "FRACA"
    else:
        interpretacao = "MUITO FRACA"
    
    direcao = "NEGATIVA (↘️)" if correlacao_tempo_churn < 0 else "POSITIVA (↗️)"
    print(f"💡 Interpretação: Correlação {interpretacao} {direcao}")
    
    # Visualizações
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. Boxplot - Tempo de contrato por grupo de churn
    dados_para_correlacao.boxplot(column=coluna_tempo, by=coluna_churn, ax=axes[0,0])
    axes[0,0].set_title(f'Boxplot: {coluna_tempo} por {coluna_churn}')
    axes[0,0].set_xlabel(f'{coluna_churn} (0=Não Cancelou, 1=Cancelou)')
    axes[0,0].set_ylabel(coluna_tempo)
    
    # 2. Histograma comparativo
    churn_0 = dados_para_correlacao[dados_para_correlacao[coluna_churn] == 0][coluna_tempo]
    churn_1 = dados_para_correlacao[dados_para_correlacao[coluna_churn] == 1][coluna_tempo]
    
    axes[0,1].hist(churn_0, bins=30, alpha=0.7, label='Não Cancelou', color='lightblue', density=True)
    axes[0,1].hist(churn_1, bins=30, alpha=0.7, label='Cancelou', color='lightcoral', density=True)
    axes[0,1].set_xlabel(coluna_tempo)
    axes[0,1].set_ylabel('Densidade')
    axes[0,1].set_title(f'Distribuição de {coluna_tempo} por Grupo')
    axes[0,1].legend()
    axes[0,1].grid(alpha=0.3)
    
    # 3. Scatter plot
    cores = ['lightblue' if x == 0 else 'lightcoral' for x in dados_para_correlacao[coluna_churn]]
    axes[1,0].scatter(dados_para_correlacao[coluna_tempo], dados_para_correlacao[coluna_churn], 
                     c=cores, alpha=0.6)
    axes[1,0].set_xlabel(coluna_tempo)
    axes[1,0].set_ylabel(coluna_churn)
    axes[1,0].set_title(f'Scatter Plot: {coluna_tempo} × {coluna_churn}')
    axes[1,0].grid(alpha=0.3)
    
    # 4. Violin plot
    dados_violin = [churn_0.values, churn_1.values]
    parts = axes[1,1].violinplot(dados_violin, positions=[0, 1], showmeans=True, showmedians=True)
    axes[1,1].set_xticks([0, 1])
    axes[1,1].set_xticklabels(['Não Cancelou', 'Cancelou'])
    axes[1,1].set_ylabel(coluna_tempo)
    axes[1,1].set_title(f'Violin Plot: {coluna_tempo} por {coluna_churn}')
    axes[1,1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Análise de segmentação por quartis de tempo
    quartis = dados_para_correlacao[coluna_tempo].quantile([0.25, 0.5, 0.75])
    print(f"\n📊 ANÁLISE POR QUARTIS DE {coluna_tempo.upper()}:")
    print("-" * 40)
    
    def classificar_quartil(valor):
        if valor <= quartis[0.25]:
            return 'Q1 (Baixo)'
        elif valor <= quartis[0.5]:
            return 'Q2 (Médio-Baixo)'
        elif valor <= quartis[0.75]:
            return 'Q3 (Médio-Alto)'
        else:
            return 'Q4 (Alto)'
    
    dados_para_correlacao['Quartil_Tempo'] = dados_para_correlacao[coluna_tempo].apply(classificar_quartil)
    quartil_analysis = dados_para_correlacao.groupby('Quartil_Tempo')[coluna_churn].agg(['count', 'sum', 'mean'])
    quartil_analysis['taxa_churn_pct'] = (quartil_analysis['mean'] * 100).round(1)
    
    print(quartil_analysis)
    
    print(f"\n💡 INSIGHTS - {coluna_tempo.upper()} × EVASÃO:")
    q1_churn = quartil_analysis.loc['Q1 (Baixo)', 'taxa_churn_pct']
    q4_churn = quartil_analysis.loc['Q4 (Alto)', 'taxa_churn_pct']
    
    if q1_churn > q4_churn:
        print(f"   • Clientes com MENOR {coluna_tempo} têm MAIOR taxa de evasão ({q1_churn}% vs {q4_churn}%)")
        print(f"   • Retenção melhora com o aumento do tempo de relacionamento")
    else:
        print(f"   • Clientes com MAIOR {coluna_tempo} têm MAIOR taxa de evasão ({q4_churn}% vs {q1_churn}%)")
        print(f"   • Padrão atípico - investigar possíveis causas")
    
    print(f"   • Diferença entre Q1 e Q4: {abs(q1_churn - q4_churn):.1f} pontos percentuais")
    
else:
    print("❌ Não foi possível realizar análise de tempo × evasão")
    print("💡 Verifique se as colunas de tempo e churn estão disponíveis")

print(f"\n✅ Análise tempo de contrato × evasão concluída!")

In [ ]:
### 💰 Análise 4: Total Gasto × Evasão

print("💰 ANÁLISE: TOTAL GASTO × EVASÃO")
print("=" * 40)

# Identificar coluna relacionada ao gasto total
coluna_gasto = None
possíveis_colunas_gasto = ['total', 'charges', 'gasto', 'valor', 'account.Charges.Total']

for col in dados_para_correlacao.columns:
    col_lower = col.lower()
    if any(termo in col_lower for termo in ['total', 'charges', 'gasto', 'valor']):
        if 'total' in col_lower:  # Priorizar variáveis com "total"
            coluna_gasto = col
            break
        elif coluna_gasto is None:  # Se não encontrou "total", aceitar outras
            coluna_gasto = col

if coluna_gasto is None:
    print("⚠️ Coluna de gasto total não encontrada automaticamente")
    print("🔍 Procurando por colunas numéricas com valores altos (possível gasto)...")
    
    for col in dados_para_correlacao.select_dtypes(include=[np.number]).columns:
        if col != coluna_churn:  # Não é a variável target
            max_val = dados_para_correlacao[col].max()
            if max_val > 1000:  # Assumir que gastos são valores altos
                coluna_gasto = col
                print(f"💡 Assumindo '{col}' como variável de gasto (max: {max_val:,.2f})")
                break

if coluna_gasto and coluna_churn:
    print(f"\n📊 ANALISANDO: {coluna_gasto} × {coluna_churn}")
    
    # Remover outliers extremos para melhor visualização
    Q1 = dados_para_correlacao[coluna_gasto].quantile(0.25)
    Q3 = dados_para_correlacao[coluna_gasto].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    dados_sem_outliers = dados_para_correlacao[
        (dados_para_correlacao[coluna_gasto] >= limite_inferior) & 
        (dados_para_correlacao[coluna_gasto] <= limite_superior)
    ]
    
    outliers_removidos = len(dados_para_correlacao) - len(dados_sem_outliers)
    print(f"📊 Outliers identificados e removidos para análise: {outliers_removidos} ({outliers_removidos/len(dados_para_correlacao)*100:.1f}%)")
    
    # Estatísticas básicas por grupo de churn
    stats_gasto = dados_para_correlacao.groupby(coluna_churn)[coluna_gasto].agg([
        'count', 'mean', 'median', 'std', 'min', 'max'
    ]).round(2)
    
    print(f"\n📈 ESTATÍSTICAS DE {coluna_gasto.upper()} POR GRUPO DE CHURN:")
    print("-" * 60)
    print(stats_gasto)
    
    # Calcular correlação específica
    correlacao_gasto_churn = dados_para_correlacao[coluna_gasto].corr(dados_para_correlacao[coluna_churn])
    print(f"\n🔗 CORRELAÇÃO {coluna_gasto} × {coluna_churn}: {correlacao_gasto_churn:+.4f}")
    
    # Interpretação da correlação
    if abs(correlacao_gasto_churn) > 0.5:
        interpretacao = "FORTE"
    elif abs(correlacao_gasto_churn) > 0.3:
        interpretacao = "MODERADA"
    elif abs(correlacao_gasto_churn) > 0.1:
        interpretacao = "FRACA"
    else:
        interpretacao = "MUITO FRACA"
    
    direcao = "NEGATIVA (↘️)" if correlacao_gasto_churn < 0 else "POSITIVA (↗️)"
    print(f"💡 Interpretação: Correlação {interpretacao} {direcao}")
    
    # Visualizações
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Boxplot - Gasto total por grupo de churn (sem outliers para melhor viz)
    dados_sem_outliers.boxplot(column=coluna_gasto, by=coluna_churn, ax=axes[0,0])
    axes[0,0].set_title(f'Boxplot: {coluna_gasto} por {coluna_churn}\n(Outliers removidos para visualização)')
    axes[0,0].set_xlabel(f'{coluna_churn} (0=Não Cancelou, 1=Cancelou)')
    axes[0,0].set_ylabel(f'{coluna_gasto}')
    
    # 2. Histograma comparativo
    churn_0_gasto = dados_para_correlacao[dados_para_correlacao[coluna_churn] == 0][coluna_gasto]
    churn_1_gasto = dados_para_correlacao[dados_para_correlacao[coluna_churn] == 1][coluna_gasto]
    
    axes[0,1].hist(churn_0_gasto, bins=30, alpha=0.7, label='Não Cancelou', color='lightblue', density=True)
    axes[0,1].hist(churn_1_gasto, bins=30, alpha=0.7, label='Cancelou', color='lightcoral', density=True)
    axes[0,1].set_xlabel(coluna_gasto)
    axes[0,1].set_ylabel('Densidade')
    axes[0,1].set_title(f'Distribuição de {coluna_gasto} por Grupo')
    axes[0,1].legend()
    axes[0,1].grid(alpha=0.3)
    
    # 3. Scatter plot com jitter para visualizar densidade
    y_jitter = dados_para_correlacao[coluna_churn] + np.random.normal(0, 0.05, len(dados_para_correlacao))
    cores = ['lightblue' if x == 0 else 'lightcoral' for x in dados_para_correlacao[coluna_churn]]
    axes[1,0].scatter(dados_para_correlacao[coluna_gasto], y_jitter, 
                     c=cores, alpha=0.6, s=20)
    axes[1,0].set_xlabel(coluna_gasto)
    axes[1,0].set_ylabel(f'{coluna_churn} (com jitter)')
    axes[1,0].set_title(f'Scatter Plot: {coluna_gasto} × {coluna_churn}')
    axes[1,0].set_ylim(-0.5, 1.5)
    axes[1,0].grid(alpha=0.3)
    
    # 4. Violin plot (usando dados sem outliers)
    dados_violin = [
        dados_sem_outliers[dados_sem_outliers[coluna_churn] == 0][coluna_gasto].values,
        dados_sem_outliers[dados_sem_outliers[coluna_churn] == 1][coluna_gasto].values
    ]
    parts = axes[1,1].violinplot(dados_violin, positions=[0, 1], showmeans=True, showmedians=True)
    axes[1,1].set_xticks([0, 1])
    axes[1,1].set_xticklabels(['Não Cancelou', 'Cancelou'])
    axes[1,1].set_ylabel(coluna_gasto)
    axes[1,1].set_title(f'Violin Plot: {coluna_gasto} por {coluna_churn}')
    axes[1,1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Análise por faixas de gasto
    # Criar faixas de gasto baseadas em quartis
    quartis_gasto = dados_para_correlacao[coluna_gasto].quantile([0, 0.25, 0.5, 0.75, 1.0])
    
    print(f"\n📊 QUARTIS DE {coluna_gasto.upper()}:")
    for i, (q, valor) in enumerate(quartis_gasto.items()):
        print(f"   Q{int(q*4) if q > 0 else 'Min'}: R$ {valor:,.2f}")
    
    def classificar_faixa_gasto(valor):
        if valor <= quartis_gasto[0.25]:
            return 'Baixo Gasto (Q1)'
        elif valor <= quartis_gasto[0.5]:
            return 'Gasto Moderado (Q2)'
        elif valor <= quartis_gasto[0.75]:
            return 'Alto Gasto (Q3)'
        else:
            return 'Muito Alto Gasto (Q4)'
    
    dados_para_correlacao['Faixa_Gasto'] = dados_para_correlacao[coluna_gasto].apply(classificar_faixa_gasto)
    faixa_analysis = dados_para_correlacao.groupby('Faixa_Gasto')[coluna_churn].agg(['count', 'sum', 'mean'])
    faixa_analysis['taxa_churn_pct'] = (faixa_analysis['mean'] * 100).round(1)
    
    print(f"\n📈 TAXA DE EVASÃO POR FAIXA DE GASTO:")
    print("-" * 50)
    print(faixa_analysis[['count', 'taxa_churn_pct']].sort_values('taxa_churn_pct', ascending=False))
    
    # Insights específicos
    print(f"\n💡 INSIGHTS - {coluna_gasto.upper()} × EVASÃO:")
    
    # Comparar diferenças entre grupos
    media_gasto_nao_churn = stats_gasto.loc[0, 'mean']
    media_gasto_churn = stats_gasto.loc[1, 'mean']
    diferenca_absoluta = abs(media_gasto_churn - media_gasto_nao_churn)
    diferenca_percentual = (diferenca_absoluta / media_gasto_nao_churn) * 100
    
    if media_gasto_churn > media_gasto_nao_churn:
        print(f"   • Clientes que CANCELARAM gastam MAIS em média (R$ {media_gasto_churn:,.2f} vs R$ {media_gasto_nao_churn:,.2f})")
        print(f"   • Diferença de R$ {diferenca_absoluta:,.2f} ({diferenca_percentual:.1f}% a mais)")
        print(f"   • Possível insatisfação com custo-benefício ou busca por alternativas mais baratas")
    else:
        print(f"   • Clientes que CANCELARAM gastam MENOS em média (R$ {media_gasto_churn:,.2f} vs R$ {media_gasto_nao_churn:,.2f})")
        print(f"   • Diferença de R$ {diferenca_absoluta:,.2f} ({diferenca_percentual:.1f}% a menos)")
        print(f"   • Possível baixo engajamento ou uso limitado dos serviços")
    
    # Análise das faixas
    maior_evasao_faixa = faixa_analysis['taxa_churn_pct'].idxmax()
    menor_evasao_faixa = faixa_analysis['taxa_churn_pct'].idxmin()
    
    print(f"   • Maior taxa de evasão: {maior_evasao_faixa} ({faixa_analysis.loc[maior_evasao_faixa, 'taxa_churn_pct']:.1f}%)")
    print(f"   • Menor taxa de evasão: {menor_evasao_faixa} ({faixa_analysis.loc[menor_evasao_faixa, 'taxa_churn_pct']:.1f}%)")
    
    variacao_faixas = faixa_analysis['taxa_churn_pct'].max() - faixa_analysis['taxa_churn_pct'].min()
    print(f"   • Variação entre faixas: {variacao_faixas:.1f} pontos percentuais")
    
else:
    print("❌ Não foi possível realizar análise de gasto × evasão")
    print("💡 Verifique se as colunas de gasto e churn estão disponíveis")

print(f"\n✅ Análise total gasto × evasão concluída!")

In [ ]:
### 🎯 Análise 5: Seleção Final de Variáveis

print("🎯 SELEÇÃO FINAL DE VARIÁVEIS PARA MODELAGEM")
print("=" * 55)

# Compilar todas as análises de correlação realizadas
print("📊 RESUMO DAS ANÁLISES DE CORRELAÇÃO:")
print("-" * 40)

if 'correlacoes_churn' in locals():
    # Classificar variáveis por importância baseada na correlação absoluta
    ranking_features = correlacoes_churn.abs().sort_values(ascending=False)
    
    print(f"🏆 RANKING COMPLETO - CORRELAÇÃO ABSOLUTA COM {coluna_churn}:")
    print("-" * 55)
    
    features_selecionadas = {'muito_importantes': [], 'importantes': [], 'moderadas': [], 'fracas': []}
    
    for i, (feature, corr_abs) in enumerate(ranking_features.items()):
        corr_original = correlacoes_churn[feature]
        
        # Classificação por nível de importância
        if corr_abs >= 0.5:
            categoria = 'muito_importantes'
            emoji = '🔴'
            nivel = 'MUITO IMPORTANTE'
        elif corr_abs >= 0.3:
            categoria = 'importantes'
            emoji = '🟠'
            nivel = 'IMPORTANTE'
        elif corr_abs >= 0.1:
            categoria = 'moderadas'
            emoji = '🟡'
            nivel = 'MODERADA'
        else:
            categoria = 'fracas'
            emoji = '🟢'
            nivel = 'FRACA'
        
        features_selecionadas[categoria].append(feature)
        
        print(f"{i+1:2d}. {emoji} {feature:25s} | r={corr_original:+.4f} | |r|={corr_abs:.4f} | {nivel}")
    
    # Resumo por categoria
    print(f"\n📈 RESUMO POR CATEGORIA DE IMPORTÂNCIA:")
    print("-" * 45)
    for categoria, features in features_selecionadas.items():
        if features:
            print(f"{categoria.replace('_', ' ').title():20s}: {len(features):2d} variáveis - {features}")
    
    # Recomendações para seleção
    total_muito_importantes = len(features_selecionadas['muito_importantes'])
    total_importantes = len(features_selecionadas['importantes'])
    total_moderadas = len(features_selecionadas['moderadas'])
    
    print(f"\n🎯 RECOMENDAÇÕES PARA SELEÇÃO DE FEATURES:")
    print("-" * 45)
    
    if total_muito_importantes > 0:
        print(f"✅ OBRIGATÓRIAS ({total_muito_importantes}): {features_selecionadas['muito_importantes']}")
    
    if total_importantes > 0:
        print(f"🔶 ALTAMENTE RECOMENDADAS ({total_importantes}): {features_selecionadas['importantes']}")
    
    if total_moderadas > 0:
        print(f"🟡 CONSIDERAR INCLUIR ({total_moderadas}): {features_selecionadas['moderadas'][:5]}")  # Top 5
        if len(features_selecionadas['moderadas']) > 5:
            print(f"   ... e mais {len(features_selecionadas['moderadas'])-5} variáveis moderadas")
    
    # Feature set recomendado
    features_recomendadas = (features_selecionadas['muito_importantes'] + 
                           features_selecionadas['importantes'] + 
                           features_selecionadas['moderadas'][:10])  # Top 10 moderadas
    
    print(f"\n🏆 CONJUNTO FINAL RECOMENDADO ({len(features_recomendadas)} variáveis):")
    print("-" * 55)
    for i, feature in enumerate(features_recomendadas, 1):
        corr_val = correlacoes_churn[feature]
        print(f"{i:2d}. {feature:25s} (r={corr_val:+.4f})")

else:
    print("⚠️ Execute a análise de correlação com target primeiro")

# Análise de multicolinearidade entre features selecionadas
if 'features_recomendadas' in locals() and len(features_recomendadas) > 1:
    print(f"\n🔍 ANÁLISE DE MULTICOLINEARIDADE - FEATURES SELECIONADAS:")
    print("-" * 55)
    
    # Matriz de correlação apenas das features selecionadas
    features_disponiveis = [f for f in features_recomendadas if f in dados_numericos.columns]
    
    if len(features_disponiveis) > 1:
        matriz_corr_selecionadas = dados_numericos[features_disponiveis].corr()
        
        # Identificar pares com alta correlação (multicolinearidade)
        pares_correlacionados = []
        for i in range(len(matriz_corr_selecionadas.columns)):
            for j in range(i+1, len(matriz_corr_selecionadas.columns)):
                corr_val = matriz_corr_selecionadas.iloc[i, j]
                if abs(corr_val) > 0.7:  # Alta correlação entre features
                    pares_correlacionados.append({
                        'var1': matriz_corr_selecionadas.columns[i],
                        'var2': matriz_corr_selecionadas.columns[j],
                        'correlacao': corr_val
                    })
        
        if pares_correlacionados:
            print("⚠️  ALTA CORRELAÇÃO ENTRE FEATURES (|r| > 0.7):")
            for par in pares_correlacionados:
                print(f"   🔴 {par['var1']} ↔ {par['var2']}: {par['correlacao']:+.3f}")
            print("💡 Considere remover uma das variáveis de cada par para evitar redundância")
        else:
            print("✅ Não há multicolinearidade significativa entre features selecionadas")
        
        # Visualização da matriz de correlação das features selecionadas
        if len(features_disponiveis) <= 15:  # Só plotar se não for muito grande
            plt.figure(figsize=(12, 10))
            mask = np.triu(np.ones_like(matriz_corr_selecionadas, dtype=bool))
            sns.heatmap(matriz_corr_selecionadas, 
                       mask=mask,
                       annot=True, 
                       fmt='.3f', 
                       center=0,
                       cmap='RdBu_r', 
                       square=True,
                       cbar_kws={'label': 'Correlação'})
            plt.title('Matriz de Correlação - Features Selecionadas', fontsize=14, fontweight='bold')
            plt.tight_layout()
            plt.show()

# Salvar informações para próximas etapas
if 'features_recomendadas' in locals():
    print(f"\n💾 VARIÁVEIS CRIADAS PARA PRÓXIMAS ETAPAS:")
    print("-" * 40)
    print(f"• features_recomendadas: Lista com {len(features_recomendadas)} variáveis selecionadas")
    print(f"• ranking_features: Ranking completo de importância")
    if 'pares_correlacionados' in locals():
        print(f"• pares_correlacionados: {len(pares_correlacionados)} pares com alta correlação")
    
    print(f"\n✅ Análise de seleção de variáveis concluída!")
    print(f"🎯 {len(features_recomendadas)} variáveis selecionadas para modelagem")
    
else:
    print("❌ Execute as análises de correlação primeiro para gerar as recomendações")

print(f"\n" + "="*60)
print("🎉 ANÁLISE COMPLETA DE CORRELAÇÃO E SELEÇÃO DE VARIÁVEIS FINALIZADA!")
print("="*60)

## 🔧 2. Preparação dos Dados para Machine Learning

Nesta etapa, vamos preparar os dados para aplicar os algoritmos de Machine Learning:

In [ ]:
# Selecionar features (variáveis independentes) e target (variável dependente)

# Remover colunas que não são úteis para o modelo
colunas_remover = ['customerID']  # ID não é predictivo

# Identificar colunas categóricas e numéricas
colunas_categoricas = dados.select_dtypes(include=['object']).columns.tolist()
colunas_numericas = dados.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Remover Churn das listas (é nossa variável target)
if 'Churn' in colunas_categoricas:
    colunas_categoricas.remove('Churn')
if 'Churn' in colunas_numericas:
    colunas_numericas.remove('Churn')

print(f"📊 Colunas categóricas: {len(colunas_categoricas)}")
print(colunas_categoricas)
print(f"\n🔢 Colunas numéricas: {len(colunas_numericas)}")
print(colunas_numericas)

# Preparar dataset final
dados_ml = dados.copy()

# Remover colunas desnecessárias
for col in colunas_remover:
    if col in dados_ml.columns:
        dados_ml = dados_ml.drop(columns=[col])

print(f"\n✅ Dataset preparado com {dados_ml.shape[1]} colunas")

In [ ]:
# Encoding de variáveis categóricas
print("=== ENCODING DE VARIÁVEIS CATEGÓRICAS ===")

dados_encoded = dados_ml.copy()

# Aplicar LabelEncoder para cada coluna categórica
label_encoders = {}

for col in colunas_categoricas:
    if col in dados_encoded.columns:
        le = LabelEncoder()
        dados_encoded[col] = le.fit_transform(dados_encoded[col].astype(str))
        label_encoders[col] = le
        
        print(f"✅ {col}: {len(le.classes_)} categorias únicas")

print(f"\n📊 Shape após encoding: {dados_encoded.shape}")

# Verificar se há valores ausentes após encoding
missing_after_encoding = dados_encoded.isnull().sum().sum()
print(f"🔍 Valores ausentes após encoding: {missing_after_encoding}")

# Remover registros com valores ausentes se houver
if missing_after_encoding > 0:
    dados_encoded = dados_encoded.dropna()
    print(f"🧹 Shape após remoção de NaN: {dados_encoded.shape}")

dados_encoded.head()

In [ ]:
# Divisão dos dados em features (X) e target (y)
X = dados_encoded.drop('Churn', axis=1)
y = dados_encoded['Churn']

print(f"📊 Features (X): {X.shape}")
print(f"🎯 Target (y): {y.shape}")
print(f"📋 Colunas features: {X.columns.tolist()}")

# Divisão em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # Manter proporção de classes
)

print(f"\n=== DIVISÃO DOS DADOS ===")
print(f"🏋️ Treino - X: {X_train.shape}, y: {y_train.shape}")
print(f"🧪 Teste - X: {X_test.shape}, y: {y_test.shape}")

# Verificar distribuição das classes no treino e teste
print(f"\n📊 Distribuição no treino:")
print(y_train.value_counts(normalize=True))
print(f"\n📊 Distribuição no teste:")
print(y_test.value_counts(normalize=True))

In [ ]:
# Normalização dos dados (StandardScaler)
print("=== NORMALIZAÇÃO DOS DADOS ===")

scaler = StandardScaler()

# Fit no conjunto de treino e transform em ambos
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"✅ Dados normalizados!")
print(f"📊 Média antes da normalização (amostra): {X_train.mean().head()}")
print(f"📊 Média após normalização (amostra): {X_train_scaled.mean(axis=0)[:5]}")
print(f"📊 Desvio padrão após normalização (amostra): {X_train_scaled.std(axis=0)[:5]}")

# Converter de volta para DataFrame (opcional, para manter nomes das colunas)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

## 🤖 3. Construção dos Modelos Preditivos

Vamos treinar e avaliar diferentes algoritmos de Machine Learning para prever o churn:

In [ ]:
# Definir os modelos para comparação
modelos = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100)
}

# Dicionário para armazenar resultados
resultados = {}

print("🚀 TREINANDO MODELOS...")

for nome_modelo, modelo in modelos.items():
    print(f"\n📊 Treinando: {nome_modelo}")
    
    # Treinar o modelo
    if nome_modelo == 'Logistic Regression':
        # Logistic Regression funciona melhor com dados normalizados
        modelo.fit(X_train_scaled, y_train)
        y_pred = modelo.predict(X_test_scaled)
        y_pred_proba = modelo.predict_proba(X_test_scaled)[:, 1]
    else:
        # Tree-based models não precisam de normalização
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)
        y_pred_proba = modelo.predict_proba(X_test)[:, 1]
    
    # Calcular métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    # Armazenar resultados
    resultados[nome_modelo] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'modelo': modelo
    }
    
    print(f"✅ {nome_modelo} treinado!")
    print(f"   Accuracy: {accuracy:.4f}")
    print(f"   F1-Score: {f1:.4f}")
    print(f"   ROC-AUC: {roc_auc:.4f}")

print("\n🎉 Todos os modelos foram treinados!")

In [ ]:
# Comparação dos modelos
print("="*60)
print("📊 COMPARAÇÃO DOS MODELOS")
print("="*60)

# Criar DataFrame com resultados
df_resultados = pd.DataFrame(resultados).T
df_metricas = df_resultados[['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']]

print(df_metricas.round(4))

# Identificar melhor modelo por métrica
print(f"\n🏆 MELHORES MODELOS POR MÉTRICA:")
for metrica in df_metricas.columns:
    melhor_modelo = df_metricas[metrica].idxmax()
    melhor_score = df_metricas[metrica].max()
    print(f"   {metrica.upper()}: {melhor_modelo} ({melhor_score:.4f})")

# Visualização dos resultados
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

metricas = ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']
colors = ['skyblue', 'lightgreen', 'coral', 'gold', 'lightpink']

for i, metrica in enumerate(metricas):
    ax = axes[i]
    df_metricas[metrica].plot(kind='bar', ax=ax, color=colors[i])
    ax.set_title(f'{metrica.replace("_", " ").title()}')
    ax.set_ylabel('Score')
    ax.set_xlabel('Modelo')
    ax.tick_params(axis='x', rotation=45)
    ax.grid(axis='y', alpha=0.3)

# Remover o subplot vazio
axes[5].remove()

plt.tight_layout()
plt.show()

## 4. Análise Detalhada do Melhor Modelo

In [ ]:
# Selecionar melhor modelo baseado no F1-Score (boa métrica para dados desbalanceados)
melhor_modelo_nome = df_metricas['f1_score'].idxmax()
melhor_modelo_resultado = resultados[melhor_modelo_nome]

print(f"🏆 MELHOR MODELO: {melhor_modelo_nome}")
print(f"📊 F1-Score: {melhor_modelo_resultado['f1_score']:.4f}")

# Matriz de Confusão
y_pred_melhor = melhor_modelo_resultado['y_pred']
cm = confusion_matrix(y_test, y_pred_melhor)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Matriz de Confusão
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Não Cancelou', 'Cancelou'],
            yticklabels=['Não Cancelou', 'Cancelou'])
axes[0].set_title(f'Matriz de Confusão - {melhor_modelo_nome}')
axes[0].set_xlabel('Predição')
axes[0].set_ylabel('Real')

# Curva ROC
y_pred_proba_melhor = melhor_modelo_resultado['y_pred_proba']
fpr, tpr, _ = roc_curve(y_test, y_pred_proba_melhor)
roc_auc = auc(fpr, tpr)

axes[1].plot(fpr, tpr, color='darkorange', lw=2, 
             label=f'ROC Curve (AUC = {roc_auc:.4f})')
axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('Taxa de Falsos Positivos')
axes[1].set_ylabel('Taxa de Verdadeiros Positivos')
axes[1].set_title('Curva ROC')
axes[1].legend(loc="lower right")
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Relatório detalhado
print(f"\n📋 RELATÓRIO DETALHADO - {melhor_modelo_nome}")
print("="*50)
print(classification_report(y_test, y_pred_melhor, 
                          target_names=['Não Cancelou', 'Cancelou']))

In [ ]:
# Análise de Importância das Features (se for Random Forest ou Decision Tree)
if melhor_modelo_nome in ['Random Forest', 'Decision Tree']:
    
    modelo_treinado = melhor_modelo_resultado['modelo']
    importancias = modelo_treinado.feature_importances_
    
    # Criar DataFrame com importâncias
    df_importancias = pd.DataFrame({
        'feature': X_train.columns,
        'importancia': importancias
    }).sort_values('importancia', ascending=False)
    
    # Plotar top 10 features mais importantes
    plt.figure(figsize=(10, 6))
    top_10 = df_importancias.head(10)
    
    plt.barh(range(len(top_10)), top_10['importancia'], color='lightcoral')
    plt.yticks(range(len(top_10)), top_10['feature'])
    plt.xlabel('Importância')
    plt.title(f'Top 10 Features Mais Importantes - {melhor_modelo_nome}')
    plt.gca().invert_yaxis()  # Maior importância no topo
    plt.tight_layout()
    plt.show()
    
    print("📊 TOP 10 FEATURES MAIS IMPORTANTES:")
    for i, row in top_10.iterrows():
        print(f"{row['feature']:30}: {row['importancia']:.4f}")

elif melhor_modelo_nome == 'Logistic Regression':
    
    modelo_treinado = melhor_modelo_resultado['modelo']
    coeficientes = modelo_treinado.coef_[0]
    
    # Criar DataFrame com coeficientes (valores absolutos para ranking)
    df_coeficientes = pd.DataFrame({
        'feature': X_train.columns,
        'coeficiente': coeficientes,
        'abs_coeficiente': np.abs(coeficientes)
    }).sort_values('abs_coeficiente', ascending=False)
    
    # Plotar top 10 coeficientes mais importantes
    plt.figure(figsize=(10, 6))
    top_10 = df_coeficientes.head(10)
    
    colors = ['red' if x < 0 else 'blue' for x in top_10['coeficiente']]
    plt.barh(range(len(top_10)), top_10['coeficiente'], color=colors)
    plt.yticks(range(len(top_10)), top_10['feature'])
    plt.xlabel('Coeficiente')
    plt.title(f'Top 10 Coeficientes Mais Importantes - {melhor_modelo_nome}')
    plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print("📊 TOP 10 COEFICIENTES MAIS IMPORTANTES:")
    for i, row in top_10.iterrows():
        sinal = "📈" if row['coeficiente'] > 0 else "📉"
        print(f"{row['feature']:30}: {row['coeficiente']:8.4f} {sinal}")

## 5. Conclusões e Recomendações

### Resultados Obtidos

In [ ]:
print("="*80)
print("📊 RELATÓRIO FINAL - MODELOS PREDITIVOS TELECOM X")
print("="*80)

# Resumo dos resultados
melhor_f1 = df_metricas['f1_score'].max()
melhor_accuracy = df_metricas['accuracy'].max()
melhor_roc_auc = df_metricas['roc_auc'].max()

print(f"\n🎯 PERFORMANCE DOS MODELOS:")
print(f"   • Melhor F1-Score: {melhor_f1:.4f} ({df_metricas['f1_score'].idxmax()})")
print(f"   • Melhor Accuracy: {melhor_accuracy:.4f} ({df_metricas['accuracy'].idxmax()})")
print(f"   • Melhor ROC-AUC: {melhor_roc_auc:.4f} ({df_metricas['roc_auc'].idxmax()})")

print(f"\n🏆 MODELO RECOMENDADO: {melhor_modelo_nome}")
print(f"   • F1-Score: {melhor_modelo_resultado['f1_score']:.4f}")
print(f"   • Accuracy: {melhor_modelo_resultado['accuracy']:.4f}")
print(f"   • Precision: {melhor_modelo_resultado['precision']:.4f}")
print(f"   • Recall: {melhor_modelo_resultado['recall']:.4f}")
print(f"   • ROC-AUC: {melhor_modelo_resultado['roc_auc']:.4f}")

print(f"\n💡 INSIGHTS PRINCIPAIS:")
print(f"   • Taxa de churn no dataset: {(y.sum()/len(y)*100):.1f}%")
print(f"   • O modelo consegue identificar {melhor_modelo_resultado['recall']:.1%} dos clientes que vão cancelar")
print(f"   • Precisão de {melhor_modelo_resultado['precision']:.1%} nas predições de churn")

print(f"\n🎯 RECOMENDAÇÕES DE NEGÓCIO:")
print(f"   1. Implementar sistema de scoring de churn usando o {melhor_modelo_nome}")
print(f"   2. Focar nas features mais importantes identificadas pelo modelo")
print(f"   3. Criar campanhas de retenção para clientes com alta probabilidade de churn")
print(f"   4. Monitorar continuamente a performance do modelo")
print(f"   5. Coletar feedback dos resultados para melhorar o modelo")

print(f"\n📈 PRÓXIMOS PASSOS:")
print(f"   • Testar técnicas de balanceamento de classes (SMOTE, undersampling)")
print(f"   • Realizar feature engineering para criar novas variáveis")
print(f"   • Experimentar outros algoritmos (XGBoost, SVM)")
print(f"   • Implementar validação cruzada mais robusta")
print(f"   • Criar pipeline de deploy do modelo")

print(f"\n" + "="*80)
print("🎉 ANÁLISE CONCLUÍDA COM SUCESSO!")
print("="*80)